In [1]:
import pandas as pd
import numpy as np
import re
from functools import reduce
import seaborn as sns
from lower_case_dict import d_dict as D_dict
from lower_case_dict import D_ocupaciones
from lower_case_dict import d_describe as D_describe
from traduce import leyenda
from traduce import traduce
import traduce as tr
pd.options.display.max_columns=None
pd.options.display.max_colwidth=300
pd.options.mode.chained_assignment=None

## Recuperación y fusión de datasets

In [2]:
# Encuesta de graduados 2019
ruta_g19='data/Graduados_19_modelos.csv'
df_g19=pd.read_csv(ruta_g19,index_col=0,low_memory=False)

In [3]:
df_g19.shape

(31651, 278)

In [4]:
# Encuesta de máster 2019
ruta_m19='data/Master_19_modelos.csv'
df_m19=pd.read_csv(ruta_m19,index_col=0,low_memory=False)

In [5]:
df_m19.shape

(11483, 279)

In [6]:
# Encuesta de grado 2014
ruta_g14='data/Grado_14_modelos.csv'
df_g14=pd.read_csv(ruta_g14,index_col=0,low_memory=False)

In [7]:
df_m19.shape

(11483, 279)

In [8]:
df=df_g19
df=df.append(df_m19,ignore_index=True)
df.shape

(43134, 290)

In [9]:
df=df.append(df_g14,ignore_index=True)
df.shape

(73513, 308)

In [10]:
# Comprobación: 
df.shape[0]==df_g19.shape[0]+df_m19.shape[0]+df_g14.shape[0]

True

In [11]:
# Inspección de columnas 
for col in df.columns:
    try: print (col, '\t', D_describe[col])
    except: print ('no', col)

año 	 Año en que se realizó la encuesta
est_encuesta 	 Estudios oficiales objeto de la encuesta (los que definen la población objeto de muestreo)
sexo 	 Mujer u hombre
edad 	 Grupo de edad en el momento de la encuesta
nacionalidad 	 Nacionalidad española u otra
eu27 	 Nacido o no en EU27
titulo_g1 	 Titulación del primer grado universitario, diplomatura, licenciatura o equivalente
rama_g1 	 Rama de conocimiento del primer titulo de grado
n_art_hum 	 Número de títulos universitarios (grado, máster, doctorado o equivalente) en la rama de artes y humanidades
n_ciencias 	 Número de títulos universitarios (grado, máster, doctorado o equivalente) en la rama de Ciencias
n_cc_soc_ju 	 Número de títulos universitarios (grado, máster, doctorado o equivalente) en la rama de ciencias sociales y jurídicas
n_ing_arq 	 Número de títulos universitarios (grado, máster, doctorado o equivalente) en la rama de ingeniería y arquitectura
n_cc_salud 	 Número de títulos universitarios (grado, máster, doctorad

### Funciones de inspección: declaración de variables globales

In [12]:
tr.D_dict=D_dict
tr.df=df

# Imputación de valores no informados

In [13]:
# Columnas de la encuesta de grado de 2019 no informadas en la encuesta de máster de 2019:
for col in df_g19.columns:
    if col not in df_m19.columns:
        print (col)

titulo_g1
varios_grados
master
ambito_g4
rama_g4
titulo_g1_
varios_grados_
master_
ambito_g4_
rama_g4_
n_master_


#### Columna 'varios_grados'

La columna 'varios_grados' (binaria) informa de si el encuestado tiene más de un título de grado universitario. 

- No está informada para la encuesta de máster de 2019. Se informa con los datos proporcionados por la columna 'n_grados', que recoge el número de grados universitarios del encuestado.

- Está parcialmente informada para la encuesta de grado de 2014: solo recoge un segundo grado universitario si se ha cursado en el extranjero. 

In [14]:
# Imputación de no informados de grado 2019 en encuesta máster 2019:
# 'varios_grados'=1 (Sí) para titulados máster con n_grados>1:
df['varios_grados'].mask((df['est_encuesta']==2) & (df['n_grados']>1),1,inplace=True)

#### Columna 'titulo_g1'

La columna 'titulo_g1' (categórica) informa de la denominación del título de grado universitario del encuestado. 

No está informada para la encuesta de máster de 2019. Se imputa el valor 1 ('No conocido') cuando el encuestado declara tener un título de grado universitario y el valor 0 ('No tiene') en caso contrario.

In [15]:
# 'titulo _g1'=1 ('No conocido') para titulados máster con grado:
df['titulo_g1'].mask((df['est_encuesta']==2) & (df['n_grados']>=1),1,inplace=True) 
# 'titulo _g1'=0 ('No tiene') para titulados máster sin grado:
df['titulo_g1'].mask((df['est_encuesta']==2) & (df['n_grados']==0),0,inplace=True)

#### Columna 'master'

La columna 'master' (binaria) informa de si el encuestado tiene algún título de máster. 

No está informada para la encuesta de máster de 2019. Se imputa el valor 1 ('Sí) a todos los encuestados porque todos participan en la encuesta por tener un título de máster.

In [16]:
# 'master'=1 (Sí) para titulados máster
df['master'].mask(df['est_encuesta']==2,1,inplace=True) 

In [17]:
# Columnas de la encuesta de máster de 2019 no informadas en la encuesta de grado de 2019:
for col in df_m19.columns:
    if col not in df_g19.columns:
        print (col)

aux_traduce
titulo_m1
titulo_m1_
ambito_m1
ambito_m1_
grado
grado_
varios_master
varios_master_
incremento
rama_m4
rama_m4_


#### Columna 'varios_master'

La columna 'varios_master' (binaria) toma valor 1 para los encuestados que declaran tener más de un título de máster universitario. 

- Se informa para la encuesta de grado de 2019 con la información proporcionada por la columna 'n_master', que recoge el número de títulos de máster del encuestado. 

- Se informa parcialmente para la encuesta de grado de 2014 con la información proporcionada por la columna 'n_master', que recoge hasta dos títulos de máster del encuestado siempre que uno haya sido cursado en el extranjero. 

In [18]:
# Imputación de no informados de máster en encuesta grado 2019:
df['varios_master'].mask((df['est_encuesta']==1) & (df['n_master']>1),1,inplace=True)
# Imputación de no informados de máster en encuesta grado 2014:
df['varios_master'].mask((df['est_encuesta']==3) & (df['n_master']>1),1,inplace=True)

#### Columnas 'titulo_m1' y 'ambito_m1'

La columna 'titulo_m1' (categórica) recoge la denominación del título de máster que hace elegibles a los entrevistados de la encuesta de máster de 2019. 

No está informada para las encuestas de grado de 2014 y de 2019. Se les imputa valor 1 ('No conocido') a quienes declaran tener un título de máster y valor 0 ('No tiene') en caso contrario. 

Se procede de forma analoga con la columna 'ambito_g1' (categórica), que recoge el ámbito de estudio del título principal de máster en la encuesta de máster de 2019.

In [19]:
# 'titulo _m1' 'No conocido' para graduados con máster:
df['titulo_m1'].mask((df['est_encuesta']==1) & (df['n_master']>0),1,inplace=True) 
# 'titulo _m1'=0 ('No tiene') para graduados sin máster:
df['titulo_m1'].mask((df['est_encuesta']==1) & (df['n_master']==0),0,inplace=True)

In [20]:
# 'ambito _m1'=1 ('No conocido') para graduados con máster:
df['ambito_m1'].mask((df['est_encuesta']==1) & (df['n_master']>0),1,inplace=True) 
# 'ambito _m1'=0 ('No tiene') para graduados sin máster:
df['ambito_m1'].mask((df['est_encuesta']==1) & (df['n_master']==0),0,inplace=True)

#### Columna 'grado'

La columna 'grado' (binaria) toma valor 1 para los encuestados con título de grado. Procede de la encuesta de máster de 2019 en la que algunos encuestados declaran no tener este título. 

No está informada para las encuestas de grado. Se imputa valor 1 a todos sus encuestados (todos tienen un título de grado universitario o equivalente). 

In [21]:
# 'grado'=1 (Sí) para graduados
df['grado'].mask(df['est_encuesta']==1,1,inplace=True) 

In [22]:
# Columnas de la encuesta de grado de 2014 no informadas en alguna de las otras dos encuestas: 
for col in df_g14.columns:
    if col not in df_g19.columns or col not in df_m19.columns:
        print (col)

titulo_g1
titulo_g1_
varios_grados
varios_grados_
master
master_
varios_master
varios_master_
incremento
mir
mir_
idioma5
idioma5_
idioma5_nivel
idioma5_nivel_
idioma6
idioma6_
idioma6_nivel
idioma6_nivel_
idioma7
idioma7_
idioma7_nivel
idioma7_nivel_
btr_spe_univ
btr_spe_univ_
etr_spe_univ
etr_spe_univ_


#### Columna 'btr_spe_univ'

La columna 'btr_spe_univ', definida en la encuesta de grado de 2014, informa de qué encuestados declaran haber buscado trabajo mediante servicios públicos de empelo o servicios de empleo de la universidad. 

En las encuestas de grado y de máster de 2019 esta información aparece desagregada en dos columnas, 'btr_spe' y 'btr_univ'. Se combinan para informar la columna 'btr_spe_univ'.

In [23]:
# Se agrupa la información sobre búsqueda de empleo mediante servicios públicos de empleo o servicios de ...
# ... empleo de la universidad para los datasets g19 y m19:
df['btr_spe_univ'][df['año']==2019]=df['btr_spe'][df['año']==2019].mask(df['btr_spe'].isna(),0)\
                                    +df['btr_univ'][df['año']==2019].mask(df['btr_univ'].isna(),0)
# Corrige la suma cuando hay unos en ambas columnas: 
df['btr_spe_univ'].mask(df['btr_spe_univ']==2,1,inplace=True)
# Corrige ceros donde ambas columnas están no informadas: 
df['btr_spe_univ'][(df['año']==2019) & df['btr_spe'].isna() & df['btr_univ'].isna()]=None

In [24]:
# Comprobación: 
df[['año','btr_spe_univ','btr_spe','btr_univ']].groupby(['año','btr_spe','btr_univ'],dropna=False)\
                                                .agg(['unique']) 

btr_spe_univ
                                unique
año  btr_spe btr_univ                 
2014 NaN     NaN       [nan, 1.0, 0.0]
2019 0.0     0.0                 [0.0]
             1.0                 [1.0]
     1.0     0.0                 [1.0]
             1.0                 [1.0]
             NaN                 [1.0]
     NaN     1.0                 [1.0]
             NaN                 [nan]

#### Columna 'etr_spe_univ'

La columna 'etr_spe_univ', definida en la encuesta de grado de 2014, informa de qué encuestados declaran haber encontrado trabajo gracias a los servicios públicos de empelo o a los servicios de empleo de la universidad.

En las encuestas de grado y de máster de 2019 esta información aparece desagregada en dos columnas, 'etr_spe' y 'etr_univ'. Se combinan para informar la columna 'etr_spe_univ'.

In [25]:
# Se agrupa la información sobre éxito en la búsqueda de empleo mediante servicios públicos o servicios de ...
# ... empleo de la universidad para los datasets g19 y m19:
df['etr_spe_univ'][df['año']==2019]=df['etr_spe'][df['año']==2019].mask(df['etr_spe'].isna(),0)\
                                    +df['etr_univ'][df['año']==2019].mask(df['etr_univ'].isna(),0)
# Corrige la suma cuando hay unos en ambas columnas: 
df['etr_spe_univ'].mask(df['etr_spe_univ']==2,1,inplace=True)
# Corrige ceros donde ambas columnas están no informadas: 
df['etr_spe_univ'][(df['año']==2019) & df['etr_spe'].isna() & df['etr_univ'].isna()]=None

In [26]:
# Comprobación: 
df[['año','etr_spe_univ','etr_spe','etr_univ']].groupby(['año','etr_spe','etr_univ'],dropna=False)\
                                                .agg(['unique']) 

etr_spe_univ
                                unique
año  etr_spe etr_univ                 
2014 NaN     NaN       [0.0, nan, 1.0]
2019 0.0     0.0                 [0.0]
             1.0                 [1.0]
     1.0     0.0                 [1.0]
             1.0                 [1.0]
     NaN     NaN                 [nan]

In [27]:
# Columnas de la encuesta de grado de 2014 no informadas en alguna de las otras dos encuestas: 
#for col in df_g19.columns.tolist()+df_m19.columns.tolist():
#    if col not in df_g14.columns:
#        print (col)

#### Número de otros títulos de estudios de grado medio o de grado superior

La encuesta de grado de 2014 solo informa de un título de grado medio y un título de grado superior de los encuestados que declaran tener alguno. Son las variables binarias 'g_medio' y 'g_sup'.

Las encuestas de grado y de máster de 2019 facilitan información de hasta 3 títulos de grado medio y de grado superior. Para estas encuestas se definieron variables ordinales que informan del número de títulos de grado medio ('n_gmedio') y de grado superior ('n_gsup'). Se informan estas columnas con los datos disponibles en la encuesta de grado de 2014.

In [28]:
# Se imputa número de otros títulos de grado medio y de grado superior en encuesta m14:
df['n_gmedio'][df['año']==2014]=df['g_medio'][df['año']==2014]
df['n_gsup'][df['año']==2014]=df['g_sup'][df['año']==2014]

In [29]:
df.shape

(73513, 308)

Algunas otras columnas no informadas en la encuesta de grado de 2014 no se pueden imputar por falta de información. 

# Nueva variable 'titulo_ppal'
'titulo_ppal' recoge en una columna única los títulos de los estudios principales de la encuesta, que corresponden distintos niveles de formación universitaria (grado y máster).  
  
Se construye a partir de la información recogida por 'titulo_g1' y 'titulo_m1': las encuestas de grado tienen 'titulo_g1' bien informado y 'titulo_m1' como binaria ('no tiene' o 'no conocido'), y viceversa para las encuestas de titulados en máster. 
  
Se crea un diccionario nuevo para el título principal a partir de la columna de códigos 'titulo_ppal' y la columna literal 'titulo_ppal_'.

In [30]:
# Códigos de títulos de grado: 
df['titulo_g1'].dropna().astype('int').sort_values().unique()

array([     0,      1,  11101,  11201,  11301,  11401,  11901,  21101,
        21201,  21301,  21302,  21401,  21502,  22201,  22202,  22301,
        22901,  23101,  23102,  23103,  23104,  23201,  23202,  23901,
        31101,  31201,  31202,  31301,  31401,  31402,  31404,  31406,
        32101,  32102,  32201,  41201,  41202,  41301,  41302,  41303,
        41401,  41402,  41403,  41601,  42101,  51101,  51201,  51202,
        51901,  52101,  53101,  53201,  53202,  53203,  53301,  54101,
        54201,  61301,  61901,  71101,  71301,  71302,  71401,  71402,
        71403,  71404,  71405,  71501,  71502,  71503,  71601,  71603,
        71901,  72101,  72102,  72201,  72401,  73101,  73102,  73201,
        73202,  81102,  81103,  81201,  82101,  84101,  91101,  91201,
        91301,  91401,  91402,  91501,  91502,  91503,  91504,  91505,
        91601,  92301, 101401, 101501, 104101, 104103, 109999])

In [31]:
# Códigos de títulos de máster: 
df['titulo_m1'].dropna().astype('int').sort_values().unique()

array([   0,    1,  111,  113,  114,  119,  211,  213,  215,  219,  221,
        222,  223,  229,  231,  232,  239,  311,  312,  313,  314,  319,
        321,  322,  411,  412,  413,  414,  416,  419,  421,  511,  512,
        519,  521,  522,  531,  532,  533,  541,  542,  599,  612,  613,
        619,  711,  712,  713,  714,  715,  716,  719,  721,  722,  731,
        732,  811,  821,  831,  841,  911,  912,  913,  914,  915,  916,
        919,  923, 1013, 1014, 1015, 1022, 1032, 1041])

### Títulos principales y títulos no informados

En las encuestas de grado el título de máster ('titulo_m1') no está bien informado; cuando el encuestado tiene alguno, figura como 'Título no conocido' (1 en 'titulo_m1'). Si no tiene, figura como 'No tiene' (0 en 'título_m1'). Para estos encuestados el título de grado está bien informado. 

De manera análoga, el título de grado ('titulo_g1') no está bien informado en las encuestas de máster y sí se conoce la denominación del título de máster.    

Al fundir los títulos de grado y de máster en una única columna ('titulo_ppal'), se conserva la denominación del titulo principal de cada encuestado (de grado o de máster según la encuesta en la que haya participado) y se se prescinde de de los valores 0 y 1 ('No tiene' o 'Título no conocido') en su título no principal. 

In [32]:
# Para encuestas de grado ('est_encuesta'=1): 'titulo_m1' binaria
df['titulo_m1'][df['est_encuesta']==1].unique()

array([0., 1.])

In [33]:
# Para encuesta de máster ('est_encuesta'=2): 'titulo_g1' binaria
df['titulo_g1'][df['est_encuesta']==2].unique()

array([1., 0.])

In [34]:
# Se prescinde de los valores 0 y 1 en 'titulo_g1' y 'titulo_m1' (todos los encuestados tienen título bien ...
# ... informado para uno de los dos niveles de formación, y solo para uno).
D_dict['titulo_g1'][0], D_dict['titulo_m1'][0], D_dict['titulo_g1'][1], D_dict['titulo_m1'][1]

('No tiene', 'No tiene', 'Título no conocido', 'Título no conocido')

In [35]:
# Para las encuestas de grado: 
df['titulo_m1'][df['titulo_m1'].isin([0,1])].count()==df[df['est_encuesta']==1].shape[0] # Comprobación

True

In [36]:
# Para las encuestas de máster: 
df['titulo_g1'][df['titulo_g1'].isin([0,1])].count()==df[df['est_encuesta']==2].shape[0] # Comprobación

True

#### Variable 'titulo_ppal' (códigos)

In [37]:
df['titulo_ppal']=df['titulo_g1']
df['titulo_ppal'].mask(df['est_encuesta']==2,df['titulo_m1'],inplace=True)
df['titulo_ppal']=df['titulo_ppal'].astype('int')
df['titulo_ppal'].unique()

array([ 11101,  11201,  11301,  11401,  11901,  21101,  21201,  21301,
        21302,  21401,  21502,  22201,  22202,  22301,  22901,  23101,
        23102,  23103,  23104,  23201,  23202,  23901,  31101,  31201,
        31202,  31301,  31401,  31402,  31404,  31406,  32101,  32102,
        32201,  41201,  41202,  41301,  41302,  41303,  41401,  41402,
        41403,  41601,  42101,  51101,  51201,  51202,  51901,  52101,
        53101,  53201,  53202,  53203,  53301,  54101,  54201,  61301,
        61901,  71101,  71301,  71302,  71401,  71402,  71403,  71404,
        71405,  71501,  71502,  71503,  71601,  71603,  71901,  72101,
        72102,  72201,  72401,  73101,  73102,  73201,  73202,  81102,
        81103,  81201,  82101,  84101,  91101,  91201,  91301,  91401,
        91402,  91501,  91502,  91503,  91504,  91505,  91601,  92301,
       101401, 101501, 104101, 104103, 109999,    916,    114,    612,
          531,    519,    915,    713,    421,   1022,    619,    716,
      

In [38]:
# Comprobación:
df['titulo_ppal'].nunique()==df[~df['titulo_g1'].isin([0,1])]['titulo_g1'].nunique()\
                            +df[~df['titulo_m1'].isin([0,1])]['titulo_m1'].nunique()

True

In [39]:
D_describe['titulo_ppal']='Código del título universitario principal '+\
                          '(el que hace a los titulados elegibles para la encuesta en la que participan)'
D_describe['titulo_ppal']

'Código del título universitario principal (el que hace a los titulados elegibles para la encuesta en la que participan)'

#### Variable 'titulo_ppal_' (denominaciones)

In [40]:
df['titulo_ppal_']=df['titulo_g1_'].where(df['est_encuesta']==1)
df['titulo_ppal_'].mask(df['est_encuesta']==2,df['titulo_m1_'],inplace=True)
#df['titulo_ppal_'].unique()

In [41]:
# Comprobación:
df['titulo_ppal_'].nunique()==df[~df['titulo_g1'].isin([0,1])]['titulo_g1_'].nunique()\
                            +df[~df['titulo_m1'].isin([0,1])]['titulo_m1_'].nunique()

True

In [42]:
D_describe['titulo_ppal_']='Denominación del título universitario principal '+\
                          '(el que hace a los titulados elegibles para la encuesta en la que participan)'
D_describe['titulo_ppal_']

'Denominación del título universitario principal (el que hace a los titulados elegibles para la encuesta en la que participan)'

#### Diccionario para 'titulo_ppal'
Se construye a partir de las columnas de códigos y de denominaciones de los títulos principales. 

In [43]:
D_dict['titulo_ppal']=dict(zip(df['titulo_ppal'].unique(),df['titulo_ppal_'].unique()))
D_dict['titulo_ppal']

{11101: 'Grado Pedagogía',
 11201: 'Grado Educación infantil',
 11301: 'Grado Educación primaria',
 11401: 'Grado Otros maestros',
 11901: 'Grado Educación social',
 21101: 'Grado Audiovisual, imagen y multimedia',
 21201: 'Grado Diseño',
 21301: 'Grado Bellas artes',
 21302: 'Grado Historia del arte',
 21401: 'Grado Conservación y restauración',
 21502: 'Grado Música y Artes escénicas',
 22201: 'Grado Arqueología',
 22202: 'Grado Historia',
 22301: 'Grado Filosofía',
 22901: 'Grado Humanidades',
 23101: 'Grado Lengua inglesa',
 23102: 'Grado Lenguas clásicas',
 23103: 'Grado Otras lenguas extranjeras',
 23104: 'Grado Traducción e interpretación',
 23201: 'Grado Lenguas y dialectos españoles',
 23202: 'Grado Literatura',
 23901: 'Grado Lenguas modernas y aplicadas',
 31101: 'Grado Economía',
 31201: 'Grado Política y gestión pública',
 31202: 'Grado Relaciones internacionales',
 31301: 'Grado Psicología',
 31401: 'Grado Antropología social y cultural y Estudios y gestión de la cultura'

In [44]:
# Comprobación: 
df['aux']=df['titulo_ppal'].replace(D_dict['titulo_ppal'])
(df['aux']==df['titulo_ppal_']).unique()

array([ True])

In [45]:
del df['aux']

# Nueva variable 'rama_ppal'

La columna 'rama_ppal' recoge la denominación de la rama de conocimiento del título principal del encuestado

Permite agregaciones de los títulos principales por ramas de conocimiento. 

In [46]:
# Para encuestas de grado ('est_encuesta'=1)
df['rama_ppal']=df['rama_g1'].where(df['est_encuesta']==1)
# Para encuesta de máster ('est_encuesta'=2)
df['rama_ppal'].mask(df['est_encuesta']==2,df['rama_m1'],inplace=True)
df['rama_ppal'].unique()

array([3., 1., 2., 4., 5., 6.])

In [47]:
# Variable literal 'rama_ppal_'
df['rama_ppal_']=df['rama_ppal'].astype('O').replace(D_dict['rama_g1'])
df['rama_ppal_'].unique()

array(['Ciencias sociales y jurídicas', 'Artes y humanidades', 'Ciencias',
       'Ingeniería y arquitectura', 'Ciencias de la salud', 'Servicios'],
      dtype=object)

# Nueva variable 'familia_ppal'  

La columna 'familia_ppal' recoge la denominación de la familia de estudios a la que pertenecen los títulos principales de los encuestados. 

Las familias de estudios universitarios se definen a partir de la clasificación en ámbito de estudios de las titulaciones de máster (menos desagregada que los ámbitos de estudio de los títulos de grado). 

Permite agregaciones de los títulos principales por familia de estudios. 

In [48]:
# Para títulos de máster, la familia principal es el ámbito de los estudios de máster: 
df['familia_ppal']=df['ambito_m1']
# De momento, la familia de los títulos de grado figura como no informada ('No tiene' o 'No conocido'):
df['familia_ppal'][df['est_encuesta']==1].unique()

array([0., 1.])

In [49]:
# Para los títulos de grado, es necesario mapear los ámbitos del título principal a la clasificación de ...
# ... ámbitos de los estudios de máster (menos detallada)
# Se mapean ambitos de grado a ámbitos de máster (familas). 
# Columna aux para identificar códigos y texto en ámbitos de grado: 
df['ambito_grado_aux']=df['ambito_g1'].astype(str)+'__'+df['ambito_g1_']
df['ambito_grado_aux'].unique().tolist()

['11.0__Educación (Otros estudios) ',
 '112.0__Formación de docentes de enseñanza infantil ',
 '113.0__Formación de docentes de enseñanza primaria ',
 '211.0__Técnicas audiovisuales y medios de comunicación ',
 '21.0__Artes (Otros estudios) ',
 '22.0__Humanidades ',
 '23.0__Lenguas ',
 '311.0__Economía ',
 '31.0__Ciencias sociales y del comportamiento (Otros estudios) ',
 '313.0__Psicología ',
 '32.0__Periodismo y documentación ',
 '41.0__Negocios y administración (Otros estudios) ',
 '413.0__Dirección y administración ',
 '42.0__Derecho ',
 '51.0__Ciencias de la vida ',
 '52.0__Medio ambiente ',
 '53.0__Ciencias químicas, físicas y geológicas ',
 '54.0__Matemáticas y estadística ',
 '61.0__Informática',
 '71.0__Ingeniería y profesiones afines ',
 '72.0__Industria manufacturera y producción ',
 '73.0__Arquitectura y construcción ',
 '81.0__Agricultura y ganadería ',
 '82.0__Silvicultura ',
 '84.0__Veterinaria ',
 '91.0__Salud (Otros estudios) ',
 '912.0__Medicina ',
 '913.0__Enfermería

In [50]:
del df['ambito_grado_aux']

In [51]:
# Correspondencia de títulos de máster y familia (ámbito de máster):
df[['titulo_m1','titulo_m1_','ambito_m1','ambito_m1_']].groupby(['ambito_m1','ambito_m1_']).agg(['unique'])

,,titulo_m1,titulo_m1_
,,unique,unique
ambito_m1,ambito_m1_,,
1.0,EDUCACIÓN,"[114.0, 111.0, 119.0, 113.0]","[Máster Formación de docentes de educación secundaria y formación profesional, Máster Ciencias de la Educación, Máster Educación (Otros estudios), Máster Formación de docentes de enseñanza primaria]"
2.0,ARTES Y HUMANIDADES,"[232.0, 223.0, 229.0, 219.0, 222.0, 211.0, 215.0, 231.0, 213.0, 239.0, 221.0]","[Máster Literatura y lingüística, Máster Filosofía y ética, Máster Humanidades (Otros estudios), Máster Artes (Otros estudios), Máster Historia y arqueología, Máster Técnicas audiovisuales y medios de comunicación, Máster Música y artes del espectáculo, Máster Aprendizaje de segundas lenguas, Má..."
3.0,"CIENCIAS SOCIALES, PERIODISMO Y DOCUMENTACIÓN","[321.0, 313.0, 314.0, 319.0, 322.0, 312.0, 311.0]","[Máster Periodismo y comunicación, Máster Psicología, Máster Sociología, antropología y geografía social y cultural, Máster Ciencias sociales y del comportamiento (Otros estudios), Máster Biblioteconomía, documentación y archivos, Máster Ciencias políticas, Máster Economía]"
4.0,"NEGOCIOS, ADMINISTRACIÓN Y DERECHO","[421.0, 412.0, 414.0, 413.0, 411.0, 416.0, 419.0]","[Máster Derecho, Máster Finanzas, banca y seguros, Máster Marketing y publicidad, Máster Dirección y administración, Máster Contabilidad y gestión de impuestos, Máster Ventas al por mayor y al por menor, Máster Negocios y administración (Otros estudios)]"
5.0,CIENCIAS,"[531.0, 519.0, 521.0, 532.0, 522.0, 533.0, 512.0, 511.0, 599.0, 541.0, 542.0]","[Máster Química, Máster Ciencias de la vida (Otros estudios), Máster Ciencias del medio ambiente, Máster Ciencias de la tierra, Máster Entornos naturales y vida silvestre, Máster Física, Máster Bioquímica, Máster Biología, Máster Ciencias químicas, físicas y geológicas; Matemáticas y estadística..."
6.0,INFORMÁTICA,"[612.0, 619.0, 613.0]","[Máster Diseño y administración de bases de datos y redes, Máster Tecnologías de la información y las comunicaciones (Otros estudios), Máster Desarrollo y análisis de aplicaciones y de software]"
7.0,"INGENIERÍA, INDUSTRIA Y CONSTRUCCIÓN","[713.0, 716.0, 712.0, 721.0, 719.0, 714.0, 722.0, 731.0, 715.0, 711.0, 732.0]","[Máster Electricidad y energía, Máster Vehículos de motor, barcos y aeronaves, Máster Control y tecnología medioambiental, Máster Industria de la alimentación, Máster Ingeniería y profesiones afines (Otros estudios), Máster Electrónica y automática, Máster Industrias de otros materiales (madera,..."
8.0,"AGRICULTURA, GANADERÍA, SILVICULTURA, PESCA Y VETERINARIA","[811.0, 831.0, 821.0, 841.0]","[Máster Producción agrícola, explotación ganadera, horticultura y jardinería, Máster Pesca, Máster Silvicultura, Máster Veterinaria]"


In [52]:
# Correspondencia de títulos de grado y ámbito de grado:
df[['titulo_g1','titulo_g1_','ambito_g1','ambito_g1_']].groupby(['ambito_g1','ambito_g1_']).agg(['unique'])

,,titulo_g1,titulo_g1_
,,unique,unique
ambito_g1,ambito_g1_,,
0.0,No tiene,[0.0],[nan]
1.0,No conocido,[1.0],[nan]
11.0,Educación (Otros estudios),"[11101.0, 11401.0, 11901.0, 1.0]","[Grado Pedagogía, Grado Otros maestros, Grado Educación social, nan]"
21.0,Artes (Otros estudios),"[21201.0, 21301.0, 21302.0, 21401.0, 21502.0, 1.0]","[Grado Diseño, Grado Bellas artes, Grado Historia del arte, Grado Conservación y restauración, Grado Música y Artes escénicas, nan]"
22.0,Humanidades,"[22201.0, 22202.0, 22301.0, 22901.0, 1.0]","[Grado Arqueología, Grado Historia, Grado Filosofía, Grado Humanidades, nan]"
23.0,Lenguas,"[23101.0, 23102.0, 23103.0, 23104.0, 23201.0, 23202.0, 23901.0, 1.0]","[Grado Lengua inglesa, Grado Lenguas clásicas, Grado Otras lenguas extranjeras, Grado Traducción e interpretación, Grado Lenguas y dialectos españoles, Grado Literatura, Grado Lenguas modernas y aplicadas, nan]"
31.0,Ciencias sociales y del comportamiento (Otros estudios),"[31201.0, 31202.0, 31401.0, 31402.0, 31404.0, 31406.0, 1.0]","[Grado Política y gestión pública, Grado Relaciones internacionales, Grado Antropología social y cultural y Estudios y gestión de la cultura, Grado Criminología, Grado Geografía, Grado Sociología e Igualdad de género, nan]"
32.0,Periodismo y documentación,"[32101.0, 32102.0, 32201.0, 1.0]","[Grado Comunicación, Grado Periodismo, Grado Información y documentación, nan]"


In [53]:
# Las familias (ámbitos de máster) se corresponden con el primer dígito de los códigos de ámbitos de ...
# ... grado excepto para los ámbitos de servicios, con código de familia de dos dígitos (10). 
# Diccionario de correspondencias (de ámbito de grado a familia):
# Solo códigos informados: 
codigos_ambito_grado=[cod for cod in df['ambito_g1'].astype(int).unique() if cod not in [0,1]]
# Para códigos de familia de un dígito: 
D_ambito_a_familia=dict(zip(codigos_ambito_grado,map(lambda cod: str(cod)[0],codigos_ambito_grado)))
# Corrección de código servicios (dos dígitos):
D_ambito_a_familia[1014]=10
D_ambito_a_familia[1015]=10
D_ambito_a_familia[109]=10
D_ambito_a_familia

{11: '1',
 112: '1',
 113: '1',
 211: '2',
 21: '2',
 22: '2',
 23: '2',
 311: '3',
 31: '3',
 313: '3',
 32: '3',
 41: '4',
 413: '4',
 42: '4',
 51: '5',
 52: '5',
 53: '5',
 54: '5',
 61: '6',
 71: '7',
 72: '7',
 73: '7',
 81: '8',
 82: '8',
 84: '8',
 91: '9',
 912: '9',
 913: '9',
 92: '9',
 1014: 10,
 1015: 10,
 109: 10}

In [54]:
# Se traduce la columna de ámbito principal ('ambito_g1') a códigos de familia ('familia_ppal'):
df['familia_ppal'].mask(df['est_encuesta']==1,df['ambito_g1'].replace(D_ambito_a_familia),inplace=True)
df['familia_ppal'][df['est_encuesta']==1].unique() # Comprobación

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.])

In [55]:
# Auxiliar para diccionario de familia principal: 
familia_ppal_codigos=df[df['est_encuesta']==2]['ambito_m1'].unique()
familia_ppal_codigos

array([ 9.,  1.,  6.,  5.,  7.,  4., 10.,  3.,  2.,  8.])

In [56]:
# Auxiliar para diccionario de familia principal: 
familia_ppal_literales=df[df['est_encuesta']==2]['ambito_m1_'].str.capitalize().unique()
familia_ppal_literales

array(['Salud y servicios sociales', 'Educación', 'Informática',
       'Ciencias', 'Ingeniería, industria y construcción',
       'Negocios, administración y derecho', 'Servicios',
       'Ciencias sociales, periodismo y documentación',
       'Artes y humanidades',
       'Agricultura, ganadería, silvicultura, pesca y veterinaria'],
      dtype=object)

In [57]:
# Diccionario de variable familia numérica a denominación literal: 
D_dict['familia_ppal']=dict(zip(familia_ppal_codigos,familia_ppal_literales))
D_dict['familia_ppal'] # Comprobación

{9.0: 'Salud y servicios sociales',
 1.0: 'Educación',
 6.0: 'Informática',
 5.0: 'Ciencias',
 7.0: 'Ingeniería, industria y construcción',
 4.0: 'Negocios, administración y derecho',
 10.0: 'Servicios',
 3.0: 'Ciencias sociales, periodismo y documentación',
 2.0: 'Artes y humanidades',
 8.0: 'Agricultura, ganadería, silvicultura, pesca y veterinaria'}

In [58]:
# Columna 'familia_ppal' literal: 
df['familia_ppal_']=df['familia_ppal'].replace(D_dict['familia_ppal'])
df['familia_ppal_'].unique() # Comprobación

array(['Educación', 'Artes y humanidades',
       'Ciencias sociales, periodismo y documentación',
       'Negocios, administración y derecho', 'Ciencias', 'Informática',
       'Ingeniería, industria y construcción',
       'Agricultura, ganadería, silvicultura, pesca y veterinaria',
       'Salud y servicios sociales', 'Servicios'], dtype=object)

# Ajuste 'extr_proxy'

La columna 'extr' (binaria) es una variable orignal de los dataset. Está referida a los estudios principales (los que hacen a los encuestados elegibles para la muestra). Informa de si se cursaron al menos en parte en el extranjero.  
  
La columna 'extr_proxy' (binaria) es una variable construida a partir de las respuestas a las preguntas sobre dónde cursó otros estudios. 
  
Se modifica la variable 'extr_proxy' para incluir a quienes cursaron al menos parte de sus estudios principales en el extranjero (si no estaban ya registrados por haber cursado otros estudios fuera de España). 

In [59]:
# Referida a los estudios que hacen elegible para la encuesta. 
D_describe['extr'], df['extr'].unique()

('Realizó parte de sus estudios en el extranjero', array([ 0.,  1., nan]))

In [60]:
# Referida al resto de estudios sobre los que se informa. 
# Creada a partir de respuestas sobre dónde cursó otros estudios.
D_describe['extr_proxy'], df['extr_proxy'].unique()

('Cursó al menos parte de sus estudios en el extranjero', array([0, 1]))

In [61]:
df['extr'].sum(), df['extr_proxy'].sum()

(9955.0, 5722)

In [62]:
df[['extr_proxy','extr']].groupby('extr').agg(['value_counts'])

extr_proxy
                value_counts
extr extr_proxy             
0.0  0                 58233
     1                  3923
1.0  0                  8157
     1                  1798

In [63]:
# Modifica 'extr_proxy' para incluir las observaciones para las que 'extr' es 1. 
df['extr_proxy'].mask(df['extr']==1,1,inplace=True)
df['extr_proxy'].sum()

13879

In [64]:
df[['extr_proxy','extr']].groupby('extr').agg(['value_counts'])

extr_proxy
                value_counts
extr extr_proxy             
0.0  0                 58233
     1                  3923
1.0  1                  9955

# Ajustes en variables de idiomas (nuevas variables)

Las encuestas de grado y de máster de 2019 informan con detalle de dos idiomas maternos y hasta cuatro idiomas extranjeros. Para los extranjeros se conoce el nivel declarado por el encuestado y el nivel acreditado por certificados oficiales. 

La encuesta de grado de 2014 informa de dos idiomas maternos y hasta 7 idiomas extranjeros. Para estos últimos, no ofrece información sobre acreditaciones oficiales del nivel de conocimiento. 

El resto de variables relacionadas con los idiomas hacen referencia a ventajas o desventajas en la búsqueda de empleo relacionadas con el conocimiento de lenguas extranjeras. 

Los idiomas están recogidos, según la encuesta, en 4 o 7 columnas nominales (categóricas), ordenadas de acuerdo con el grado de conocimiento que declara el encuestado. Así, las columnas no se corresponden con distintas lenguas, sino que cada columna incluye todos los idiomas que los encuestados han mencionado en la posición correspondiente. 

Se desagrega esta información, construyendo nuevas columnas binarias, una para cada idioma, que informan de si el encuestado declara conocerla o no. Y se crean nuevas columnas ordinales para cada idioma que informa del nivel de conocimiento declarado y del acreditado para cada encuestado. 

#### Lenguas cooficiales

En las encuestas de grado y de máster de 2019 las lenguas cooficiales de España figuran individualizadas. En la encuesta de grado de 2014, figuran todas agrupadas bajo la categoría 'lengua_cooficial'.

In [65]:
df['idioma_m1_'][df['año']==2019].unique()

array(['Español', 'Catalán', 'Euskera', 'Inglés', 'Alemán', 'Valenciano',
       'Gallego', 'Portugués', 'Otros idiomas', 'Árabe', 'Francés',
       'Italiano', 'Rumano', 'Ruso', 'Chino'], dtype=object)

In [66]:
df['idioma_m1_'][df['año']==2014].unique()

array(['Español', 'lengua_cooficial', nan, 'Inglés', 'Otros idiomas',
       'Francés', 'Ruso', 'Árabe', 'Portugués', 'Alemán', 'Italiano',
       'Chino'], dtype=object)

In [67]:
# Inspección: columnas referidas a idiomas.
col_idiomas=list(filter(lambda col: re.findall('.*idioma.*',col),df.columns))
col_idiomas

['idioma_m1',
 'idioma_m2',
 'n_idiomas',
 'idioma1',
 'idioma1_nivel',
 'idioma1_acred',
 'idioma2',
 'idioma2_nivel',
 'idioma2_acred',
 'idioma3',
 'idioma3_nivel',
 'idioma3_acred',
 'idioma4',
 'idioma4_nivel',
 'idioma4_acred',
 'dbtr_idiomas',
 'vbtr_idiomas',
 'idioma_m1_',
 'idioma_m2_',
 'idioma1_',
 'idioma1_nivel_',
 'idioma1_acred_',
 'idioma2_',
 'idioma2_nivel_',
 'idioma2_acred_',
 'idioma3_',
 'idioma3_nivel_',
 'idioma3_acred_',
 'idioma4_',
 'idioma4_nivel_',
 'idioma4_acred_',
 'dbtr_idiomas_',
 'vbtr_idiomas_',
 'idioma5',
 'idioma5_',
 'idioma5_nivel',
 'idioma5_nivel_',
 'idioma6',
 'idioma6_',
 'idioma6_nivel',
 'idioma6_nivel_',
 'idioma7',
 'idioma7_',
 'idioma7_nivel',
 'idioma7_nivel_']

In [68]:
check_list=['idioma_m1_','idioma_m2_','idioma1_','idioma2_','idioma3_','idioma4_','idioma5_','idioma6_',
            'idioma7_' ]

# Se comprueba que ningún encuestado (fila) menciona el mismo idioma en dos columnas de idioma distintas: 
for col_a in check_list:
    for col_b in [col for col in check_list if col!=col_a]:
        print (df[col_a][df[col_a]==df[col_b]].unique())
    print ('_________________________________')
# Las repeticiones solo se dan en los agregados 'otros idiomas' y 'lengua cooficial'

['Otros idiomas' 'lengua_cooficial']
['Otros idiomas' 'lengua_cooficial']
['Otros idiomas' 'lengua_cooficial']
['Otros idiomas']
['Otros idiomas']
['Otros idiomas']
[]
[]
_________________________________
['Otros idiomas' 'lengua_cooficial']
['Otros idiomas' 'lengua_cooficial']
['Otros idiomas' 'lengua_cooficial']
['Otros idiomas']
['Otros idiomas']
[]
['Otros idiomas']
[]
_________________________________
['Otros idiomas' 'lengua_cooficial']
['Otros idiomas' 'lengua_cooficial']
['Otros idiomas' 'lengua_cooficial']
['Otros idiomas' 'lengua_cooficial']
['Otros idiomas' 'lengua_cooficial']
[]
['lengua_cooficial']
['lengua_cooficial']
_________________________________
['Otros idiomas' 'lengua_cooficial']
['Otros idiomas' 'lengua_cooficial']
['Otros idiomas' 'lengua_cooficial']
['Otros idiomas' 'lengua_cooficial']
['Otros idiomas']
[]
['lengua_cooficial']
[]
_________________________________
['Otros idiomas']
['Otros idiomas']
['Otros idiomas' 'lengua_cooficial']
['Otros idiomas' 'lengua_c

In [69]:
# Inspección: primera lengua materna: contiene observaciones de todos los idiomas contemplados en la encuesta
df['idioma_m1_'].value_counts()

Español             61707
lengua_cooficial     3860
Catalán              3254
Gallego               970
Euskera               938
Valenciano            759
Otros idiomas         185
Inglés                101
Francés                77
Portugués              53
Árabe                  50
Alemán                 49
Rumano                 45
Ruso                   42
Italiano               40
Chino                  27
Name: idioma_m1_, dtype: int64

In [70]:
df['idioma_m1_'].nunique()

16

In [71]:
# Inspección: segunda lengua materna: 
df['idioma_m2_'].unique()

array([nan, 'Catalán', 'Gallego', 'Español', 'Inglés', 'Otros idiomas',
       'Euskera', 'Valenciano', 'Francés', 'Árabe', 'Alemán', 'Italiano',
       'Portugués', 'Ruso', 'Chino', 'Rumano', 'lengua_cooficial'],
      dtype=object)

In [72]:
# Inspección: primer idioma extranjero (siete idiomas extranjeros: 'idioma1' a 'idioma7'): 
df['idioma1_'].unique()

array(['Inglés', nan, 'Catalán', 'Francés', 'Italiano', 'Euskera',
       'Alemán', 'Español', 'Gallego', 'Portugués', 'Otros idiomas',
       'Valenciano', 'Árabe', 'Chino', 'Ruso', 'Rumano',
       'lengua_cooficial'], dtype=object)

In [73]:
# Inspección: nivel de conocimiento del idioma declarado por el encuestado: 
df['idioma1_nivel_'].unique()

array(['Nivel medio (Entiende lo esencial en el lenguaje común y redacta textos sencillos)',
       'Nivel alto (Entiende una gran variedad de textos complicados y usa el idioma con flexibilidad)',
       nan, 'Nivel bajo (Entiende y usa las expresiones más comunes )'],
      dtype=object)

In [74]:
# Inspección: nivel de conocimiento del idioma acreditado: 
df['idioma1_acred_'].unique()

array(['B2 (FCE, DELF B2, ZDfB, …)', 'B1 (PET, DELF B1, ZD, …)',
       'C1 (CAE, DALF C1, PWD, …)', nan,
       'A2 (DELF A2, Fit in Deutsch 2, …)', 'C2 (CPE, DALF C2, ZOP, …)',
       'A1 (DELF A1, Fit in Deutsch 1, ….)'], dtype=object)

#### Idiomas maternos:

In [75]:
# Auxiliares para comprobaciones (conserva las variables que se transforman con pd.get_dummies): 
df['materno_1']=df['idioma_m1_']
df['materno_2']=df['idioma_m2_']

In [76]:
# Columnas binarias para idiomas maternos: 
df=pd.get_dummies(data=df,columns=['idioma_m1_','idioma_m2_'],prefix=['m1','m2'])

In [77]:
# Se prenscinde de las columnas referidas a 'Otros_idiomas' (poco informativa)
del df['m1_Otros idiomas'], df['m2_Otros idiomas']

In [78]:
# Se prenscinde de las columnas referidas a 'lengua_cooficial' (requiere tratamiento diferenciado)
del df['m1_lengua_cooficial'], df['m2_lengua_cooficial']

In [79]:
# Inspección: 
df.columns[-(df['idioma_m1'].nunique()-1)*2:]

Index(['materno_1', 'materno_2', 'm1_Alemán', 'm1_Catalán', 'm1_Chino',
       'm1_Español', 'm1_Euskera', 'm1_Francés', 'm1_Gallego', 'm1_Inglés',
       'm1_Italiano', 'm1_Portugués', 'm1_Rumano', 'm1_Ruso', 'm1_Valenciano',
       'm1_Árabe', 'm2_Alemán', 'm2_Catalán', 'm2_Chino', 'm2_Español',
       'm2_Euskera', 'm2_Francés', 'm2_Gallego', 'm2_Inglés', 'm2_Italiano',
       'm2_Portugués', 'm2_Rumano', 'm2_Ruso', 'm2_Valenciano', 'm2_Árabe'],
      dtype='object')

In [80]:
# lista de columnas de idioma materno (aux.):
m1_m2=list(filter(lambda col:col.startswith('m1_') or col.startswith('m2_'),df.columns))
#m1_m2

In [81]:
# Lista de idiomas maternos registrados en columnas binarias: 
idiomas=list(set(map(lambda string: string.split('_')[1],m1_m2)))
idiomas

['Ruso',
 'Chino',
 'Alemán',
 'Inglés',
 'Italiano',
 'Euskera',
 'Español',
 'Gallego',
 'Valenciano',
 'Árabe',
 'Francés',
 'Rumano',
 'Portugués',
 'Catalán']

In [82]:
# Columnas binarias para cada idioma materno: se informa por las dos columnas para idiomas maternos m1_idioma ...
# ... y m2_idioma.
maternos=list()
for idioma in idiomas:
    df[str(idioma)+'_materno']=df['m1_'+str(idioma)]+df['m2_'+str(idioma)]
    maternos.append(str(idioma)+'_materno')
df[maternos].agg(['unique'])

,Ruso_materno,Chino_materno,Alemán_materno,Inglés_materno,Italiano_materno,Euskera_materno,Español_materno,Gallego_materno,Valenciano_materno,Árabe_materno,Francés_materno,Rumano_materno,Portugués_materno,Catalán_materno
unique,"[0, 1]","[0, 1]","[0, 1]","[0, 1]","[0, 1]","[0, 1]","[1, 0]","[0, 1]","[0, 1]","[0, 1]","[0, 1]","[0, 1]","[0, 1]","[0, 1]"


In [83]:
# Comprobación:
df[maternos].sum(), df['materno_1'].value_counts()+df['materno_2'].value_counts() 

(Ruso_materno             68
 Chino_materno            41
 Alemán_materno          141
 Inglés_materno          996
 Italiano_materno         98
 Euskera_materno        1791
 Español_materno       69442
 Gallego_materno        2712
 Valenciano_materno     2224
 Árabe_materno            90
 Francés_materno         346
 Rumano_materno           52
 Portugués_materno       114
 Catalán_materno        5487
 dtype: int64,
 Alemán                141
 Catalán              5487
 Chino                  41
 Español             69442
 Euskera              1791
 Francés               346
 Gallego              2712
 Inglés                996
 Italiano               98
 Otros idiomas         486
 Portugués             114
 Rumano                 52
 Ruso                   68
 Valenciano           2224
 lengua_cooficial     8839
 Árabe                  90
 dtype: int64)

#### Idiomas extranjeros: 

In [84]:
# Una colección de dummies para cada idioma extranjero (idioma1 a idioma7):
df=pd.get_dummies(data=df,columns=['idioma1_','idioma2_','idioma3_','idioma4_','idioma5_','idioma6_','idioma7_'],
                  prefix=['i1','i2','i3','i4','i5','i6','i7'])

In [85]:
# Se prescinde de columnas referidas a 'Otros_idiomas' y a 'lengua_cooficial'
for n in range(1,8):
    del df['i'+str(n)+'_Otros idiomas']
    del df['i'+str(n)+'_lengua_cooficial']

In [86]:
# listas_idiomas=lista de listas de columnas dummy para cada idioma extranjero:
listas_idiomas=[list(filter(lambda col:col.startswith('i'+str(n)+'_'),df.columns)) for n in range(1,8)]
#listas_idiomas

In [87]:
# Inspección: lista de primeros idiomas extranjeros (i1_idioma) y valores únicos en col. binarias.
listas_idiomas[0], df[listas_idiomas[0]].agg(['unique'])

(['i1_Alemán',
  'i1_Catalán',
  'i1_Chino',
  'i1_Español',
  'i1_Euskera',
  'i1_Francés',
  'i1_Gallego',
  'i1_Inglés',
  'i1_Italiano',
  'i1_Portugués',
  'i1_Rumano',
  'i1_Ruso',
  'i1_Valenciano',
  'i1_Árabe'],
        i1_Alemán i1_Catalán i1_Chino i1_Español i1_Euskera i1_Francés  \
 unique    [0, 1]     [0, 1]   [0, 1]     [0, 1]     [0, 1]     [0, 1]   
 
        i1_Gallego i1_Inglés i1_Italiano i1_Portugués i1_Rumano i1_Ruso  \
 unique     [0, 1]    [1, 0]      [0, 1]       [0, 1]    [0, 1]  [0, 1]   
 
        i1_Valenciano i1_Árabe  
 unique        [0, 1]   [0, 1]  )

In [88]:
# Convierte listas_idiomas (lista de listas) en una lista única de idiomas (sin agrupar en sublistas)
lista_idiomas_unica=list(reduce(lambda lista1,lista2: lista1+lista2, listas_idiomas))

In [89]:
df[lista_idiomas_unica].agg(['value_counts'])

,i1_Alemán,i1_Catalán,i1_Chino,i1_Español,i1_Euskera,i1_Francés,i1_Gallego,i1_Inglés,i1_Italiano,i1_Portugués,i1_Rumano,i1_Ruso,i1_Valenciano,i1_Árabe,i2_Alemán,i2_Catalán,i2_Chino,i2_Español,i2_Euskera,i2_Francés,i2_Gallego,i2_Inglés,i2_Italiano,i2_Portugués,i2_Rumano,i2_Ruso,i2_Valenciano,i2_Árabe,i3_Alemán,i3_Catalán,i3_Chino,i3_Español,i3_Euskera,i3_Francés,i3_Gallego,i3_Inglés,i3_Italiano,i3_Portugués,i3_Rumano,i3_Ruso,i3_Valenciano,i3_Árabe,i4_Alemán,i4_Catalán,i4_Chino,i4_Español,i4_Euskera,i4_Francés,i4_Gallego,i4_Inglés,i4_Italiano,i4_Portugués,i4_Rumano,i4_Ruso,i4_Valenciano,i4_Árabe,i5_Alemán,i5_Chino,i5_Francés,i5_Italiano,i5_Portugués,i5_Ruso,i5_Árabe,i6_Alemán,i6_Chino,i6_Francés,i6_Italiano,i6_Portugués,i6_Ruso,i6_Árabe,i7_Chino,i7_Italiano,i7_Ruso,i7_Árabe
,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts
0,73088,72519,73496,71274,72947,70770,73287,17850,72685,73051,73507,73496,72833,73488,70505,72918,73392,73305,73204,61225,73352,65963,70538,71844,73498,73445,73157,73438,72079,73285,73378,73477,73435,71413,73449,72895,72088,72731,73502,73422,73418,73421,73252,73447,73458,73503,73500,73248,73497,73461,73113,73324,73509,73466,73497,73471,73497,73501,73505,73496,73485,73506,73509,73511,73510,73512,73508,73509,73507,73511,73512,73512,73512,73512
1,425,994,17,2239,566,2743,226,55663,828,462,6,17,680,25,3008,595,121,208,309,12288,161,7550,2975,1669,15,68,356,75,1434,228,135,36,78,2100,64,618,1425,782,11,91,95,92,261,66,55,10,13,265,16,52,400,189,4,47,16,42,16,12,8,17,28,7,4,2,3,1,5,4,6,2,1,1,1,1


### Nuevas variables: 'i', con i={idiomas observados}
Una columna binaria por idioma, con valor 1 cuando el encuestado conoce el idioma, ya sea como lengua materna o como idioma extranjero aprendido.   
  
No tiene en cuenta el nivel de conocimiento del idioma ni su acreditación oficial. Por tanto, tiene valores informados siempre que el encuestado menciona el idioma, incluso cuando no informa de su nivel de conocimiento o de acreditación. 

In [90]:
# Se reunen todas las columnas referidas al mismo idioma, sea como lengua materna o como lengua extranjera: 
for idioma in idiomas: 
    # asignación inicial: información sobre cada idioma como lengua materna: 
    df[str(idioma)]=df[str(idioma)+'_materno']
    # Se añade la info sobre el idioma declarado como extranjero: 
    for n in range(1,8): 
        try: df[str(idioma)]=df[str(idioma)]+df['i'+str(n)+'_'+str(idioma)] 
        # evita traceback cuando no existe la columna dummy (sin observaciones del idioma en alguna col. ...
        # ... de idioma original)
        except: pass 

In [91]:
# Inspección: 
df[[str(idioma) for idioma in idiomas]].agg(['value_counts'])

,Ruso,Chino,Alemán,Inglés,Italiano,Euskera,Español,Gallego,Valenciano,Árabe,Francés,Rumano,Portugués,Catalán
,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts
0,73208,73128,68226,8634,67764,70756,1578,70334,70142,73182,55762,73425,70265,66143
1,305,385,5287,64879,5749,2757,71935,3179,3371,331,17751,88,3248,7370


### Contador de repeticiones
Se construye un **contador de repeticiones de valores iguales en una selección de columnas de un dataframe**.  
  
Permite conocer, para cada valor presente en la selección, cuantas observaciones tienen ese valor repetido en las distintas columnas para cada número de repeticiones concebible.  
   
Aquí se emplea para comprobar que el resultado cuadra con los registros resultantes de la tranformación de las columnas asociadas a los idiomas. 

In [92]:
def une_listas(L1,L2):  
    l1=list(set(L1))
    return l1+[x for x in L2 if x not in l1]

In [93]:
def formato(valor):
    try: valor=int(valor)
    except: pass
    return valor

In [94]:
def cuenta_repeticiones(data,columns=None): 
    from functools import reduce
    if columns==None: 
        columns=data.columns
    df_aux=data[columns]
    cols=df_aux.columns
    valores_brutos=list(map(lambda col: df_aux[col].unique().tolist(),cols))
    valores=list(reduce(une_listas,valores_brutos))
    valores_notna=[v for v in valores if v==v] # prescinde de nan
    valores_notna=list(map(formato,valores_notna))
    D_out=dict(map(lambda v: (v,df_aux[df_aux==v].count(axis=1)),valores_notna))
    if len(valores)!=len(valores_notna):
        D_out['nan']=len(cols)-df_aux[cols].count(axis=1)
    dx=pd.DataFrame(D_out)
    N=range(len(cols)+1)
    D=dict(map(lambda v: (v,list(map(lambda n: dx[v][dx[v]==n].count(),N))),dx.columns))
    out=pd.DataFrame(D)
    out.index.name='Repetitions'
    return out

In [95]:
check=cuenta_repeticiones(df,['idioma_m1','idioma_m2','idioma1','idioma2','idioma3','idioma4','idioma5','idioma6',
                              'idioma7'])
check.rename(columns=D_dict['idioma1'])

,Español,Catalán,Euskera,Gallego,Valenciano,Inglés,Francés,Alemán,Italiano,Portugués,Rumano,Árabe,Chino,Ruso,Otros idiomas,lengua_cooficial,nan
Repetitions,,,,,,,,,,,,,,,,,
0,1578,66143,70756,70334,70142,8634,55762,68226,67764,70265,73425,73182,73128,73208,70986,62948,6
1,71935,7370,2757,3179,3371,64879,17751,5287,5749,3248,88,331,385,305,2361,10438,15
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,150,126,56
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14,1,540
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,2794
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10747
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,27413
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26450
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4136


In [96]:
# Comprobación: los resultados cuadran con los calculados mediante cuenta_repeticiones:  
df[[str(idioma) for idioma in idiomas]].agg(['value_counts'])

,Ruso,Chino,Alemán,Inglés,Italiano,Euskera,Español,Gallego,Valenciano,Árabe,Francés,Rumano,Portugués,Catalán
,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts
0,73208,73128,68226,8634,67764,70756,1578,70334,70142,73182,55762,73425,70265,66143
1,305,385,5287,64879,5749,2757,71935,3179,3371,331,17751,88,3248,7370


In [97]:
# Diccionarios asociados a las variables 'i' (i={idiomas observados})
for idioma in idiomas: 
    D_describe[str(idioma)]='Conoce o no el idioma (binaria)'
    D_dict[str(idioma)]=dict()
    D_dict[str(idioma)][0]='No'
    D_dict[str(idioma)][1]='Sí'

In [98]:
# Variables literales para 'i' (i={idiomas}):
for idioma in idiomas: 
    df[str(idioma)+'_']=df[str(idioma)].replace(D_dict[str(idioma)])
    print (idioma, df[str(idioma)+'_'].unique())

Ruso ['No' 'Sí']
Chino ['No' 'Sí']
Alemán ['No' 'Sí']
Inglés ['Sí' 'No']
Italiano ['No' 'Sí']
Euskera ['No' 'Sí']
Español ['Sí' 'No']
Gallego ['No' 'Sí']
Valenciano ['No' 'Sí']
Árabe ['No' 'Sí']
Francés ['No' 'Sí']
Rumano ['No' 'Sí']
Portugués ['No' 'Sí']
Catalán ['No' 'Sí']


### Nuevas variables: 'i_nivel', con i={idiomas observados}
Una columna por idioma, graduada de 0 a 3 según el nivel de conocimiento declarado por el encuestado. Se imputa nivel alto (valor 3) a los hablantes nativos del idioma.  
  
No se tienen en cuenta los registros para los que el nivel de conocimiento del idioma no está informado. Por ello, tiene menos valores informados que las variables de idioma 'i' (i={idiomas}).
  
**Nota**: **Se invierte el sentido de la escala** respecto a la definida originalmente. Esto es, se asocian los niveles más altos de conocimiento del idioma a valores numéricos más altos y se asocia el valor 0 a no conocer el idioma. 

In [99]:
# Nuevas variables de idiomas graduados por nivel 
df['idioma1_nivel_'].unique() # Inspección

array(['Nivel medio (Entiende lo esencial en el lenguaje común y redacta textos sencillos)',
       'Nivel alto (Entiende una gran variedad de textos complicados y usa el idioma con flexibilidad)',
       nan, 'Nivel bajo (Entiende y usa las expresiones más comunes )'],
      dtype=object)

In [100]:
# Escala numérica orginal (se invierte en las escalas definidas para las variables nuevas)
leyenda('idioma1_nivel')

1 	 Nivel alto (Entiende una gran variedad de textos complicados y usa el idioma con flexibilidad)
2 	 Nivel medio (Entiende lo esencial en el lenguaje común y redacta textos sencillos)
3 	 Nivel bajo (Entiende y usa las expresiones más comunes )
9 	 NS/NC
  	 No aplicable


In [101]:
# Se prescinde de los registros de idiomas  para los que el nivel de conocimiento no está informado:
for i in range(1,8):
    infs_idioma=df['idioma'+str(i)].count()
    infs_nivel=df['idioma'+str(i)+'_nivel'].count()
    dif=infs_idioma-infs_nivel
    print (f'idioma {i}: Informados idioma: {infs_idioma}, informados nivel: {infs_nivel}, omitidos: {dif}')

idioma 1: Informados idioma: 66321, informados nivel: 65997, omitidos: 324
idioma 2: Informados idioma: 30825, informados nivel: 30696, omitidos: 129
idioma 3: Informados idioma: 8026, informados nivel: 7988, omitidos: 38
idioma 4: Informados idioma: 1745, informados nivel: 1738, omitidos: 7
idioma 5: Informados idioma: 141, informados nivel: 141, omitidos: 0
idioma 6: Informados idioma: 42, informados nivel: 42, omitidos: 0
idioma 7: Informados idioma: 12, informados nivel: 12, omitidos: 0


#### Para los idiomas maternos:

In [102]:
# Para los idiomas maternos: se imputa nivel alto (3) 
for materno in maternos: 
    df[materno]=df[materno]*3
df[maternos].agg(['value_counts'])

,Ruso_materno,Chino_materno,Alemán_materno,Inglés_materno,Italiano_materno,Euskera_materno,Español_materno,Gallego_materno,Valenciano_materno,Árabe_materno,Francés_materno,Rumano_materno,Portugués_materno,Catalán_materno
,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts
0,73445,73472,73372,72517,73415,71722,4071,70801,71289,73423,73167,73461,73399,68026
3,68,41,141,996,98,1791,69442,2712,2224,90,346,52,114,5487


#### Para los idiomas extranjeros:

In [103]:
# Variables de nivel auxiliares para invertir el sentido de la escala: 
for n in range(1,8):
    df['nivel_aux_i'+str(n)]=df['idioma'+str(n)+'_nivel'].replace({1:3,3:1,None:0})
df[['idioma1_nivel','nivel_aux_i1']].groupby('idioma1_nivel').agg(['unique']) # Comprobación: 

,nivel_aux_i1
,unique
idioma1_nivel,
1.0,[3.0]
2.0,[2.0]
3.0,[1.0]


In [104]:
# Nivel declarado de conocimiento del idioma: 
# Se multiplican las variables dummy por el nivel de conocimento declarado de cada idioma extranjero. 
for n in range(7):
    for idioma in listas_idiomas[n]:
        df[idioma+'_nivel']=df[idioma]*df['nivel_aux_i'+str(n+1)] # nivel de acuerdo con la escala redefinida

In [105]:
# lista_idiomas_nivel =lista de variables referidas al nivel de cada idioma
lista_idiomas_nivel=list(map(lambda col: col+'_nivel',lista_idiomas_unica))
df[lista_idiomas_nivel].agg(['value_counts']) # Inspección

,i1_Alemán_nivel,i1_Catalán_nivel,i1_Chino_nivel,i1_Español_nivel,i1_Euskera_nivel,i1_Francés_nivel,i1_Gallego_nivel,i1_Inglés_nivel,i1_Italiano_nivel,i1_Portugués_nivel,i1_Rumano_nivel,i1_Ruso_nivel,i1_Valenciano_nivel,i1_Árabe_nivel,i2_Alemán_nivel,i2_Catalán_nivel,i2_Chino_nivel,i2_Español_nivel,i2_Euskera_nivel,i2_Francés_nivel,i2_Gallego_nivel,i2_Inglés_nivel,i2_Italiano_nivel,i2_Portugués_nivel,i2_Rumano_nivel,i2_Ruso_nivel,i2_Valenciano_nivel,i2_Árabe_nivel,i3_Alemán_nivel,i3_Catalán_nivel,i3_Chino_nivel,i3_Español_nivel,i3_Euskera_nivel,i3_Francés_nivel,i3_Gallego_nivel,i3_Inglés_nivel,i3_Italiano_nivel,i3_Portugués_nivel,i3_Rumano_nivel,i3_Ruso_nivel,i3_Valenciano_nivel,i3_Árabe_nivel,i4_Alemán_nivel,i4_Catalán_nivel,i4_Chino_nivel,i4_Español_nivel,i4_Euskera_nivel,i4_Francés_nivel,i4_Gallego_nivel,i4_Inglés_nivel,i4_Italiano_nivel,i4_Portugués_nivel,i4_Rumano_nivel,i4_Ruso_nivel,i4_Valenciano_nivel,i4_Árabe_nivel,i5_Alemán_nivel,i5_Chino_nivel,i5_Francés_nivel,i5_Italiano_nivel,i5_Portugués_nivel,i5_Ruso_nivel,i5_Árabe_nivel,i6_Alemán_nivel,i6_Chino_nivel,i6_Francés_nivel,i6_Italiano_nivel,i6_Portugués_nivel,i6_Ruso_nivel,i6_Árabe_nivel,i7_Chino_nivel,i7_Italiano_nivel,i7_Ruso_nivel,i7_Árabe_nivel
,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts
0.0,73088,72527,73496,71296,72953,70780,73292,18106,72688,73055,73507.0,73496,72842,73488,70516,72923,73394,73306.0,73205,61265,73352,66011,70551,71848,73498,73445,73160,73438,72089,73285,73378,73477.0,73435,71423,73449,72899,72094,72733,73502,73423,73419,73421,73253,73448,73458.0,73503.0,73500,73248,73497,73461,73116,73324,73509,73466,73497,73471,73497.0,73501.0,73505.0,73496,73485,73506.0,73509.0,73511.0,73510.0,73512.0,73508.0,73509.0,73507.0,73511.0,73512.0,73512.0,73512.0,73512.0
1.0,62,39,9,6,49,537,14,3704,70,57,NaN,3,33,2,1218,92,65,NaN,100,4917,36,739,677,447,3,31,50,43,810,52,94,NaN,37,973,18,76,510,244,6,55,25,49,170,19,42.0,2.0,8,139,4,5,159,87,2,24,6,29,10.0,9.0,6.0,7,9,2.0,4.0,1.0,3.0,NaN,4.0,2.0,4.0,1.0,NaN,NaN,1.0,1.0
2.0,107,153,5,27,152,1051,51,26200,266,177,1.0,3,216,13,1270,218,49,6.0,135,5269,53,3791,1375,773,6,24,139,23,485,100,34,1.0,27,884,27,323,656,377,4,29,33,33,71,29,13.0,NaN,2,108,4,22,184,77,1,18,4,10,6.0,NaN,2.0,6,16,5.0,NaN,1.0,NaN,1.0,1.0,2.0,2.0,NaN,1.0,1.0,NaN,NaN
3.0,256,794,3,2184,359,1145,156,25503,489,224,5.0,11,422,10,509,280,5,201.0,73,2062,72,2972,910,445,6,13,164,9,129,76,7,35.0,14,233,19,215,253,159,1,6,36,10,19,17,NaN,8.0,3,18,8,25,54,25,1,5,6,3,NaN,3.0,NaN,4,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN


In [106]:
# Se reunen todas las columnas de nivel referidas al mismo idioma: 
for idioma in idiomas: 
    # Asignación inicial: nivel de lenguas declaradas como maternas
    df[str(idioma)+'_nivel']=df[str(idioma)+'_materno']
    # Se actualiza con nivel declarado de idiomas extranjeros
    for n in range(1,8): 
        try: df[str(idioma)+'_nivel']=df[str(idioma)+'_nivel']+df['i'+str(n)+'_'+str(idioma)+'_nivel']  
        except: pass # evita traceback cuando no hay columna de nivel para el idioma en alguna posición

In [107]:
# Inspección: 
df[[str(idioma)+'_nivel' for idioma in idiomas]].agg(['value_counts'])

,Ruso_nivel,Chino_nivel,Alemán_nivel,Inglés_nivel,Italiano_nivel,Euskera_nivel,Español_nivel,Gallego_nivel,Valenciano_nivel,Árabe_nivel,Francés_nivel,Rumano_nivel,Portugués_nivel,Catalán_nivel
,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts
0.0,73209,73130,68248,8942,67789,70763,1601,70339,70155,73182,55822,73425,70275,66157
1.0,120,222,2271,4524,1427,194,8,72,114,129,6572,11,846,202
2.0,81,102,1940,30336,2489,316,34,135,392,79,7315,12,1422,500
3.0,103,59,1054,29711,1808,2240,71870,2967,2852,123,3804,65,970,6654


In [108]:
# Diccionarios asociados a las variables 'i_nivel' (i={idiomas registrados})
for idioma in idiomas: 
    D_describe[str(idioma)+'_nivel']='Nivel de conocimiento del idioma declarado por el encuestado'
    D_dict[str(idioma)+'_nivel']=dict()
    D_dict[str(idioma)+'_nivel'][0]='Desconocido' # No conoce el idioma o el nivel no está informado
    D_dict[str(idioma)+'_nivel'][1]='Nivel bajo'
    D_dict[str(idioma)+'_nivel'][2]='Nivel medio'
    D_dict[str(idioma)+'_nivel'][3]='Nivel alto'

In [109]:
# Variables literales para 'i_nivel' (i={idiomas}):
for idioma in idiomas: 
    df[str(idioma)+'_nivel_']=df[str(idioma)+'_nivel'].replace(D_dict[str(idioma)+'_nivel'])
    print (idioma, df[str(idioma)+'_nivel_'].unique())

Ruso ['Desconocido' 'Nivel bajo' 'Nivel medio' 'Nivel alto']
Chino ['Desconocido' 'Nivel bajo' 'Nivel medio' 'Nivel alto']
Alemán ['Desconocido' 'Nivel bajo' 'Nivel medio' 'Nivel alto']
Inglés ['Nivel medio' 'Nivel alto' 'Desconocido' 'Nivel bajo']
Italiano ['Desconocido' 'Nivel medio' 'Nivel alto' 'Nivel bajo']
Euskera ['Desconocido' 'Nivel medio' 'Nivel alto' 'Nivel bajo']
Español ['Nivel alto' 'Desconocido' 'Nivel medio' 'Nivel bajo']
Gallego ['Desconocido' 'Nivel alto' 'Nivel medio' 'Nivel bajo']
Valenciano ['Desconocido' 'Nivel medio' 'Nivel alto' 'Nivel bajo']
Árabe ['Desconocido' 'Nivel medio' 'Nivel alto' 'Nivel bajo']
Francés ['Desconocido' 'Nivel medio' 'Nivel bajo' 'Nivel alto']
Rumano ['Desconocido' 'Nivel alto' 'Nivel medio' 'Nivel bajo']
Portugués ['Desconocido' 'Nivel medio' 'Nivel alto' 'Nivel bajo']
Catalán ['Desconocido' 'Nivel alto' 'Nivel medio' 'Nivel bajo']


### Nuevas variables: 'i_acred', con i={idiomas observados}
Una columna por idioma, graduada de 0 a 6 según el nivel de conocimiento acreditado con título oficial.  
  
No se tienen en cuenta los registros para los que el nivel de acreditación de conocimiento del idioma no está informado. Por ello, tiene menos valores informados que las variables de idioma 'i' (i={idiomas}).   

No está informado para las observaciones procedentes de la encuesta de grado de 2014. Por ello, no hay información sobre acreditación para los idiomas en las columnas 5 a 7. 
  
Se imputa nivel C1 a los hablantes nativos del idioma.

In [110]:
leyenda('idioma1_acred')

1 	 A1 (DELF A1, Fit in Deutsch 1, ….)
2 	 A2 (DELF A2, Fit in Deutsch 2, …)
3 	 B1 (PET, DELF B1, ZD, …)
9 	 NS/NC
  	 No_aplicable
4 	 B2 (FCE, DELF B2, ZDfB, …)
5 	 C1 (CAE, DALF C1, PWD, …)
6 	 C2 (CPE, DALF C2, ZOP, …)


In [111]:
df['idioma1_acred_'].unique() # Inspección

array(['B2 (FCE, DELF B2, ZDfB, …)', 'B1 (PET, DELF B1, ZD, …)',
       'C1 (CAE, DALF C1, PWD, …)', nan,
       'A2 (DELF A2, Fit in Deutsch 2, …)', 'C2 (CPE, DALF C2, ZOP, …)',
       'A1 (DELF A1, Fit in Deutsch 1, ….)'], dtype=object)

In [112]:
# Se abrevian los registros literales de las variables referidas al nivel de acreditación: 
for n in range(1,5):
    df['idioma'+str(n)+'_acred_']=df['idioma'+str(n)+'_acred_'].str.split(' ',1).str[0]
df['idioma3_acred_'].unique() # Inspección

array([nan, 'B2', 'B1', 'A2', 'C1', 'A1', 'C2'], dtype=object)

In [113]:
# Se prescinde de los registros de idiomas para los que la acreditación no está informada:
for i in range(1,5):
    infs_idioma=df['idioma'+str(i)].count()
    infs_nivel=df['idioma'+str(i)+'_acred'].count()
    dif=infs_idioma-infs_nivel
    print (f'idioma {i}: Informados idioma: {infs_idioma}, informados nivel: {infs_nivel}, omitidos: {dif}')

idioma 1: Informados idioma: 66321, informados nivel: 23494, omitidos: 42827
idioma 2: Informados idioma: 30825, informados nivel: 7707, omitidos: 23118
idioma 3: Informados idioma: 8026, informados nivel: 1733, omitidos: 6293
idioma 4: Informados idioma: 1745, informados nivel: 355, omitidos: 1390


#### Para los idiomas maternos:

In [114]:
# Para los idiomas maternos: se imputa nivel C1 (valor 5)
for materno in maternos: 
    df[materno]=df[materno]*5/3 # (deshace la imputación por nivel (valor 3) y asigna acreditación C1 (valor 5))
df[maternos].agg(['value_counts'])

,Ruso_materno,Chino_materno,Alemán_materno,Inglés_materno,Italiano_materno,Euskera_materno,Español_materno,Gallego_materno,Valenciano_materno,Árabe_materno,Francés_materno,Rumano_materno,Portugués_materno,Catalán_materno
,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts
0.0,73445,73472,73372,72517,73415,71722,4071,70801,71289,73423,73167,73461,73399,68026
5.0,68,41,141,996,98,1791,69442,2712,2224,90,346,52,114,5487


#### Para los idiomas extranjeros:

In [115]:
df[lista_idiomas_unica].agg(['value_counts'])

,i1_Alemán,i1_Catalán,i1_Chino,i1_Español,i1_Euskera,i1_Francés,i1_Gallego,i1_Inglés,i1_Italiano,i1_Portugués,i1_Rumano,i1_Ruso,i1_Valenciano,i1_Árabe,i2_Alemán,i2_Catalán,i2_Chino,i2_Español,i2_Euskera,i2_Francés,i2_Gallego,i2_Inglés,i2_Italiano,i2_Portugués,i2_Rumano,i2_Ruso,i2_Valenciano,i2_Árabe,i3_Alemán,i3_Catalán,i3_Chino,i3_Español,i3_Euskera,i3_Francés,i3_Gallego,i3_Inglés,i3_Italiano,i3_Portugués,i3_Rumano,i3_Ruso,i3_Valenciano,i3_Árabe,i4_Alemán,i4_Catalán,i4_Chino,i4_Español,i4_Euskera,i4_Francés,i4_Gallego,i4_Inglés,i4_Italiano,i4_Portugués,i4_Rumano,i4_Ruso,i4_Valenciano,i4_Árabe,i5_Alemán,i5_Chino,i5_Francés,i5_Italiano,i5_Portugués,i5_Ruso,i5_Árabe,i6_Alemán,i6_Chino,i6_Francés,i6_Italiano,i6_Portugués,i6_Ruso,i6_Árabe,i7_Chino,i7_Italiano,i7_Ruso,i7_Árabe
,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts
0,73088,72519,73496,71274,72947,70770,73287,17850,72685,73051,73507,73496,72833,73488,70505,72918,73392,73305,73204,61225,73352,65963,70538,71844,73498,73445,73157,73438,72079,73285,73378,73477,73435,71413,73449,72895,72088,72731,73502,73422,73418,73421,73252,73447,73458,73503,73500,73248,73497,73461,73113,73324,73509,73466,73497,73471,73497,73501,73505,73496,73485,73506,73509,73511,73510,73512,73508,73509,73507,73511,73512,73512,73512,73512
1,425,994,17,2239,566,2743,226,55663,828,462,6,17,680,25,3008,595,121,208,309,12288,161,7550,2975,1669,15,68,356,75,1434,228,135,36,78,2100,64,618,1425,782,11,91,95,92,261,66,55,10,13,265,16,52,400,189,4,47,16,42,16,12,8,17,28,7,4,2,3,1,5,4,6,2,1,1,1,1


In [116]:
# Nivel acreditado del idioma: 
# Se multiplican las variables dummy por el nivel de acreditación de cada idioma extranjero. 
for n in range(4):
    for idioma in listas_idiomas[n]:
        df[idioma]=df[idioma]*df['idioma'+str(n+1)+'_acred'].mask(df['idioma'+str(n+1)+'_acred'].isna(),0)
df[lista_idiomas_unica].agg(['value_counts']) # Inspección

,i1_Alemán,i1_Catalán,i1_Chino,i1_Español,i1_Euskera,i1_Francés,i1_Gallego,i1_Inglés,i1_Italiano,i1_Portugués,i1_Rumano,i1_Ruso,i1_Valenciano,i1_Árabe,i2_Alemán,i2_Catalán,i2_Chino,i2_Español,i2_Euskera,i2_Francés,i2_Gallego,i2_Inglés,i2_Italiano,i2_Portugués,i2_Rumano,i2_Ruso,i2_Valenciano,i2_Árabe,i3_Alemán,i3_Catalán,i3_Chino,i3_Español,i3_Euskera,i3_Francés,i3_Gallego,i3_Inglés,i3_Italiano,i3_Portugués,i3_Rumano,i3_Ruso,i3_Valenciano,i3_Árabe,i4_Alemán,i4_Catalán,i4_Chino,i4_Español,i4_Euskera,i4_Francés,i4_Gallego,i4_Inglés,i4_Italiano,i4_Portugués,i4_Rumano,i4_Ruso,i4_Valenciano,i4_Árabe,i5_Alemán,i5_Chino,i5_Francés,i5_Italiano,i5_Portugués,i5_Ruso,i5_Árabe,i6_Alemán,i6_Chino,i6_Francés,i6_Italiano,i6_Portugués,i6_Ruso,i6_Árabe,i7_Chino,i7_Italiano,i7_Ruso,i7_Árabe
,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts
0.0,73353,72927,73504.0,73121,73127,72803,73411,53161,73279,73389,73511.0,73509.0,73116,73506.0,72595,73305,73466,73469.0,73366,71314,73482,70647,72915,73244,73510.0,73496.0,73339,73506.0,73091,73452,73473.0,73506.0,73487.0,73104,73504.0,73268,73288,73425,73512.0,73494.0,73471.0,73486,73436,73499.0,73490.0,73512.0,73508.0,73460.0,73511.0,73485.0,73458,73494.0,73512.0,73506.0,73506.0,73499.0,73497.0,73501.0,73505.0,73496.0,73485.0,73506.0,73509.0,73511.0,73510.0,73512.0,73508.0,73509.0,73507.0,73511.0,73512.0,73512.0,73512.0,73512.0
1.0,9,9,4.0,13,2,19,4,147,6,4,NaN,NaN,18,NaN,87,9,6,4.0,5,121,1,23,34,11,NaN,NaN,4,NaN,59,5,7.0,NaN,3.0,40,NaN,1,18,12,NaN,5.0,NaN,5,12,NaN,6.0,NaN,1.0,4.0,NaN,NaN,8,NaN,NaN,2.0,NaN,3.0,16.0,12.0,8.0,17.0,28.0,7.0,4.0,2.0,3.0,1.0,5.0,4.0,6.0,2.0,1.0,1.0,1.0,1.0
2.0,11,16,1.0,8,10,47,1,413,22,6,NaN,NaN,29,NaN,183,14,13,NaN,13,364,3,72,86,55,1.0,5.0,15,2.0,120,3,19.0,NaN,4.0,89,NaN,6,37,14,1.0,6.0,2.0,8,23,3.0,3.0,NaN,1.0,13.0,NaN,NaN,9,5.0,NaN,2.0,1.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,24,30,1.0,16,49,155,11,5488,68,39,NaN,NaN,60,4.0,286,26,16,3.0,40,641,1,732,193,88,NaN,6.0,34,2.0,123,11,6.0,1.0,6.0,116,1.0,72,79,23,NaN,7.0,11.0,7,19,1.0,11.0,NaN,1.0,18.0,NaN,5.0,14,5.0,NaN,2.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.0,48,60,1.0,34,112,250,15,8025,83,39,NaN,NaN,46,NaN,227,44,8,1.0,47,711,7,1125,191,68,1.0,4.0,31,1.0,83,12,6.0,1.0,7.0,118,2.0,101,56,25,NaN,1.0,7.0,2,14,2.0,2.0,NaN,NaN,13.0,1.0,13.0,16,7.0,1.0,1.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,53,335,2.0,134,181,150,39,5164,42,20,1.0,2.0,194,1.0,107,85,3,14.0,38,283,13,723,80,40,1.0,1.0,75,2.0,29,22,2.0,3.0,6.0,31,4.0,50,21,9,NaN,NaN,16.0,2,7,6.0,NaN,NaN,1.0,5.0,1.0,8.0,6,2.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6.0,15,136,NaN,187,32,89,32,1115,13,16,1.0,2.0,50,2.0,28,30,1,22.0,4,79,6,191,14,7,NaN,1.0,15,NaN,8,8,NaN,2.0,NaN,15,2.0,15,14,5,NaN,NaN,6.0,3,2,2.0,1.0,1.0,1.0,NaN,NaN,2.0,2,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [117]:
# Se reunen todas las columnas referidas al mismo idioma: 
for idioma in idiomas: 
    df[str(idioma)+'_acred']=df[str(idioma)+'_materno']
    for n in range(1,5): 
        df[str(idioma)+'_acred']=df[str(idioma)+'_acred']+df['i'+str(n)+'_'+str(idioma)]

In [118]:
# Comprobación: 
df[[str(idioma)+'_acred' for idioma in idiomas]].agg(['value_counts'])

,Ruso_acred,Chino_acred,Alemán_acred,Inglés_acred,Italiano_acred,Euskera_acred,Español_acred,Gallego_acred,Valenciano_acred,Árabe_acred,Francés_acred,Rumano_acred,Portugués_acred,Catalán_acred
,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts
0.0,73398,73353,71795,49026,72303,71158,3627,70657,70669,73368,69796,73454.0,72899,67157
1.0,7,23,167,171,66,11,17,5,22,8,184,NaN,27,23
2.0,13,36,337,491,154,28,8,4,47,16,513,2.0,80,36
3.0,15,34,452,6297,354,96,20,13,105,15,930,NaN,155,68
4.0,6,17,372,9264,346,166,36,25,87,6,1092,2.0,139,118
5.0,71,48,337,6941,247,2017,69593,2769,2511,95,815,54.0,185,5935
6.0,3,2,53,1323,43,37,212,40,72,5,183,1.0,28,176


In [119]:
# Diccionarios asociados a las variables 'i_acred' (i={idiomas observados})
for idioma in idiomas: 
    D_describe[str(idioma)+'_acred']='Nivel acreditado del idioma'
    D_dict[str(idioma)+'_acred']=dict()
    D_dict[str(idioma)+'_acred'][0]='No' # Sin acreditación informada
    D_dict[str(idioma)+'_acred'][1]='A1'
    D_dict[str(idioma)+'_acred'][2]='A2'
    D_dict[str(idioma)+'_acred'][3]='B1'
    D_dict[str(idioma)+'_acred'][4]='B2'
    D_dict[str(idioma)+'_acred'][5]='C1'
    D_dict[str(idioma)+'_acred'][6]='C2'

In [120]:
# Variables literales para 'i_acred' (i={idiomas}):
for idioma in idiomas: 
    df[str(idioma)+'_acred_']=df[str(idioma)+'_acred'].replace(D_dict[str(idioma)+'_acred'])
    print (idioma, df[str(idioma)+'_acred_'].unique())

Ruso ['No' 'A2' 'B1' 'C1' 'A1' 'C2' 'B2']
Chino ['No' 'B1' 'A1' 'A2' 'B2' 'C1' 'C2']
Alemán ['No' 'B2' 'A2' 'B1' 'C1' 'A1' 'C2']
Inglés ['B2' 'B1' 'C1' 'No' 'A2' 'C2' 'A1']
Italiano ['No' 'B1' 'B2' 'A2' 'C1' 'A1' 'C2']
Euskera ['No' 'C1' 'B2' 'B1' 'A2' 'C2' 'A1']
Español ['C1' 'No' 'C2' 'A1' 'B1' 'B2' 'A2']
Gallego ['No' 'C1' 'A2' 'B2' 'C2' 'B1' 'A1']
Valenciano ['No' 'B1' 'C1' 'A1' 'A2' 'B2' 'C2']
Árabe ['No' 'C1' 'B1' 'A2' 'A1' 'B2' 'C2']
Francés ['No' 'B2' 'A2' 'B1' 'C1' 'A1' 'C2']
Rumano ['No' 'C1' 'C2' 'B2' 'A2']
Portugués ['No' 'B2' 'A2' 'B1' 'C2' 'C1' 'A1']
Catalán ['No' 'C1' 'B1' 'B2' 'C2' 'A1' 'A2']


In [121]:
# Se prescinde de columnas auxiliares: 
del df['materno_1'], df['materno_2']
for idioma in idiomas: 
    del df['m1_'+str(idioma)], df['m2_'+str(idioma)]
    del df[str(idioma)+'_materno']
    for n in range(1,8):
        try: del df['nivel_aux_i'+str(n)] # solo uno por cada valor de n
        except: pass # a partir del segundo idioma del ciclo
        try: 
            del df['i'+str(n)+'_'+str(idioma)]
            del df['i'+str(n)+'_'+str(idioma)+'_nivel']
        except:pass

In [122]:
# Se prescinde de columnas de idiomas originales que han sido transformadas
del df['idioma_m1'], df['idioma_m2']
for n in range(1,8):
    del df['idioma'+str(n)]
    del df['idioma'+str(n)+'_nivel'], df['idioma'+str(n)+'_nivel_']
    if n<5:
        del df['idioma'+str(n)+'_acred'], df['idioma'+str(n)+'_acred_']

In [123]:
# Inspección: columnas referidas a idiomas en el dataset: 
col_idiomas=list(filter(lambda col:re.findall('.*idioma.*',col),df.columns))
for idioma in idiomas:
    for elemento in filter(lambda col:re.findall(str(idioma)+'.*',col),df.columns):
        col_idiomas.append(elemento)
#col_idiomas

In [124]:
# Se reescriben los nombres de columnas referidas a idiomas: en minúsculas 
for col in col_idiomas:
    df.rename(columns={col:col.lower()},inplace=True)
Idiomas=idiomas
idiomas=list()
for idioma in Idiomas: 
    idiomas.append(idioma.lower())
idiomas

['ruso',
 'chino',
 'alemán',
 'inglés',
 'italiano',
 'euskera',
 'español',
 'gallego',
 'valenciano',
 'árabe',
 'francés',
 'rumano',
 'portugués',
 'catalán']

In [125]:
# Inspección: columnas referidas a idiomas.
col_idiomas=list(filter(lambda col:re.findall('.*idioma.*',col),df.columns))
for idioma in idiomas:
    for elemento in filter(lambda col:re.findall(str(idioma)+'.*',col),df.columns):
        col_idiomas.append(elemento)
col_idiomas

['n_idiomas',
 'dbtr_idiomas',
 'vbtr_idiomas',
 'dbtr_idiomas_',
 'vbtr_idiomas_',
 'ruso',
 'ruso_',
 'ruso_nivel',
 'ruso_nivel_',
 'ruso_acred',
 'ruso_acred_',
 'chino',
 'chino_',
 'chino_nivel',
 'chino_nivel_',
 'chino_acred',
 'chino_acred_',
 'alemán',
 'alemán_',
 'alemán_nivel',
 'alemán_nivel_',
 'alemán_acred',
 'alemán_acred_',
 'inglés',
 'inglés_',
 'inglés_nivel',
 'inglés_nivel_',
 'inglés_acred',
 'inglés_acred_',
 'italiano',
 'italiano_',
 'italiano_nivel',
 'italiano_nivel_',
 'italiano_acred',
 'italiano_acred_',
 'euskera',
 'euskera_',
 'euskera_nivel',
 'euskera_nivel_',
 'euskera_acred',
 'euskera_acred_',
 'español',
 'español_',
 'español_nivel',
 'español_nivel_',
 'español_acred',
 'español_acred_',
 'gallego',
 'gallego_',
 'gallego_nivel',
 'gallego_nivel_',
 'gallego_acred',
 'gallego_acred_',
 'valenciano',
 'valenciano_',
 'valenciano_nivel',
 'valenciano_nivel_',
 'valenciano_acred',
 'valenciano_acred_',
 'árabe',
 'árabe_',
 'árabe_nivel',
 'árab

## Ajuste variables asociadas a familias de estudios de grado medio

Se agrupan todas las familias de estudios de grado medio en **variables ordinales**, de modo que cada columna recoja el número de estudios de grado medio de esa familia en los que está titulado el encuestado. 

No se dispone de información sobre familias de estudios de grado medio para los encuestados en la encuesta de grado de 2014.

In [126]:
# Inspección: 
df[['fam_gmedio1','fam_gmedio1_']].groupby('fam_gmedio1').agg(['value_counts'])

,,fam_gmedio1_
,,value_counts
fam_gmedio1,fam_gmedio1_,
0.0,No tiene,41791
1.0,No conocido,38
2.0,Marítimo-pesquera,2
3.0,Industrias alimentarias,13
4.0,Química,13
5.0,Imagen personal,29
6.0,Sanidad,230
7.0,Seguridad y medio ambiente,2


In [127]:
# Inspección:
df['fam_gmedio2_'].unique()

array(['No tiene', 'Sanidad ', 'Imagen personal ',
       'Administración y gestión ',
       'Servicios socioculturales y a la comunidad ',
       'Comercio y marketing',
       'Actividades físicas y deportivas (incluidas enseñanzas deportivas de grado medio)',
       'Transporte y mantenimiento de vehículos ',
       'Enseñanzas artísticas profesionales', 'Imagen y sonido',
       'Hostelería y turismo ', 'Industrias alimentarias ',
       'Electricidad y electrónica', 'Artes gráficas ',
       'Instalación y mantenimiento', 'Edificación y obra civil ',
       'Química', nan], dtype=object)

In [128]:
# Inspección:
df['fam_gmedio3_'].unique()

array(['No tiene', 'Sanidad ',
       'Servicios socioculturales y a la comunidad ',
       'Actividades físicas y deportivas (incluidas enseñanzas deportivas de grado medio)',
       nan], dtype=object)

In [129]:
# Comprobación: todas las familias en la segunda y la tercera columnas originales están presentes también en ...
# ... la primera (permite tomar la primera como base para las agregaciones de familias)
df['fam_gmedio2'].isin(df['fam_gmedio1']).unique(), df['fam_gmedio3'].isin(df['fam_gmedio1']).unique()

(array([ True]), array([ True]))

In [130]:
# Variables binarias para cada familia de estudios de grado medio en cada posición (de 1 a 3):
df=pd.get_dummies(data=df,columns=['fam_gmedio1_','fam_gmedio2_','fam_gmedio3_'],prefix=['fgm1','fgm2','fgm3'])

In [131]:
# Renombra las columnas resultantes de get_dummies prescindiendo del los textos entre paréntesis: 
D_fix=dict()
for col in list(filter(lambda col: col.startswith('fgm'),df.columns)): 
    D_fix[col]=col.split('(')[0].strip()
df.rename(columns=D_fix,inplace=True)

In [132]:
# Se prescinde de las columnas asociadas a valores 0 y 1 ('No tiene' o 'No conocido')
del df['fgm1_No tiene'], df['fgm2_No tiene'], df['fgm3_No tiene'], df['fgm1_No conocido']
try: df['fgm1_No conocido']
except: pass
try: df['fgm1_No conocido']
except: pass

In [133]:
# Lista de columnas binarias asociadas a familias de estudios de grado medio en la columna 'fam_gmedio1'...
# ... (contiene todas las familias registradas):
lista_fam_gmed_1=list(filter(lambda col: col.startswith('fgm1'),df.columns))   
lista_fam_gmed_1[:5]

['fgm1_Actividades físicas y deportivas',
 'fgm1_Administración y gestión',
 'fgm1_Artes gráficas',
 'fgm1_Comercio y marketing',
 'fgm1_Edificación y obra civil']

In [134]:
# Lista de familias de estudios de grado medio: 
lista_familias=list(map(lambda familia: familia.split('_')[1],lista_fam_gmed_1))
lista_familias[:5]

['Actividades físicas y deportivas',
 'Administración y gestión',
 'Artes gráficas',
 'Comercio y marketing',
 'Edificación y obra civil']

In [135]:
# Se recogen los valores asociados a la misma familia en todas las columnas (hasta 3) asociadas a la misma: 
for familia in lista_familias:
    df['g_medio: '+str(familia)]=df['fgm1_'+str(familia)]
    if 'fgm2_'+str(familia) in df.columns: 
        df['g_medio: '+str(familia)]=df['g_medio: '+str(familia)]+df['fgm2_'+str(familia)]
    if 'fgm3_'+str(familia) in df.columns: 
        df['g_medio: '+str(familia)]=df['g_medio: '+str(familia)]+df['fgm3_'+str(familia)]

In [136]:
col_familias_gmed=list()
for familia in lista_familias: 
    for col in list(filter(lambda col: re.findall( 'g_medio: '+str(familia),col),df.columns)):
        col_familias_gmed.append(col)
#col_familias_gmed

In [137]:
# Inspección: 
df[col_familias_gmed].agg(['value_counts'])

,g_medio: Actividades físicas y deportivas,g_medio: Administración y gestión,g_medio: Artes gráficas,g_medio: Comercio y marketing,g_medio: Edificación y obra civil,g_medio: Electricidad y electrónica,g_medio: Enseñanzas artísticas profesionales,g_medio: Fabricación mecánica,g_medio: Hostelería y turismo,g_medio: Imagen personal,g_medio: Imagen y sonido,g_medio: Industrias alimentarias,g_medio: Industrias extractivas,g_medio: Informática y comunicaciones,g_medio: Instalación y mantenimiento,"g_medio: Madera, mueble y corcho",g_medio: Marítimo-pesquera,g_medio: Química,g_medio: Sanidad,g_medio: Seguridad y medio ambiente,g_medio: Servicios socioculturales y a la comunidad,"g_medio: Textil, confección y piel",g_medio: Transporte y mantenimiento de vehículos
,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts
0,73319,73306.0,73503.0,73486.0,73503.0,73426.0,73183.0,73491.0,73478.0,73478.0,73492.0,73500.0,73512.0,73500.0,73502.0,73512.0,73511.0,73499.0,73272,73511.0,73467.0,73506.0,73493.0
1,178,205.0,8.0,27.0,9.0,85.0,320.0,22.0,32.0,32.0,21.0,12.0,1.0,13.0,11.0,1.0,2.0,14.0,227,2.0,46.0,7.0,20.0
2,15,2.0,2.0,NaN,1.0,2.0,10.0,NaN,3.0,3.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,12,NaN,NaN,NaN,NaN
3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN


In [138]:
# Comprobación: 
check=cuenta_repeticiones(df,['fam_gmedio1','fam_gmedio2','fam_gmedio3'])
check.rename(columns=D_dict['fam_gmedio1'])

,No tiene,Agraria,Marítimo-pesquera,Industrias alimentarias,Química,Imagen personal,Sanidad,Seguridad y medio ambiente,Fabricación mecánica,Instalación y mantenimiento,Electricidad y electrónica,Transporte y mantenimiento de vehículos,Industrias extractivas,Edificación y obra civil,"Madera, mueble y corcho","Textil, confección y piel",Artes gráficas,Imagen y sonido,Informática y comunicaciones,Administración y gestión,Comercio y marketing,Servicios socioculturales y a la comunidad,Hostelería y turismo,Actividades físicas y deportivas (incluidas enseñanzas deportivas de grado medio),Enseñanzas artísticas profesionales,nan
Repetitions,,,,,,,,,,,,,,,,,,,,,,,,,,
0,30386,73475,73511,73500,73499,73478,73272,73511,73491,73502,73426,73493,73512,73503,73512,73506,73503,73492,73500,73306,73486,73467,73478,73319,73183,43134
1,87,38,2,12,14,32,227,2,22,11,85,20,1,9,1,7,8,21,13,205,27,46,32,178,320,0
2,1249,0,0,1,0,3,12,0,0,0,2,0,0,1,0,0,2,0,0,2,0,0,3,15,10,0
3,41791,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,30379


In [139]:
# Actualiza listas de columnas auxiliares para eliminarlas: 
lista_fam_gmed_1=list(filter(lambda col: col.startswith('fgm1'),df.columns))  
lista_fam_gmed_2=list(filter(lambda col: col.startswith('fgm2'),df.columns)) 
lista_fam_gmed_3=list(filter(lambda col: col.startswith('fgm3'),df.columns)) 

In [140]:
# Se prescinde de columnas auxiliares: 
for col in lista_fam_gmed_1+lista_fam_gmed_2+lista_fam_gmed_3:
    del df[col]

In [141]:
# Actualización de diccionarios: 
for familia in lista_familias: 
    D_describe['g_medio: '+str(familia)]='Núm. de títulos de la familia de estudios de grado medio '+str(familia)
    #print (D_describe['g_medio: '+str(familia)])

## Ajuste variables asociadas a familias de estudios de grado superior

Se agrupan todas las familias de estudios de grado superior en **variables ordinales**, de modo que cada columna recoja el número de estudios de grado superior de esa familia en los que está titulado el encuestado.  

No se dispone de información sobre familias de estudios de grado superior para los encuestados en la encuesta de grado de 2014.
   
El proceso de transformación es análogo al seguido para las familas de estudios de grado medio. 

In [142]:
# Inspección: 
df[['fam_gsup1','fam_gsup1_']].groupby('fam_gsup1').agg(['value_counts'])

,,fam_gsup1_
,,value_counts
fam_gsup1,fam_gsup1_,
0.0,No tiene,37877
1.0,No conocido,199
2.0,Marítimo-pesquera,24
3.0,Industrias alimentarias,36
4.0,Química,128
5.0,Imagen personal,62
6.0,Sanidad,785
7.0,Seguridad y medio ambiente,65


In [143]:
# Inspección:
df['fam_gsup2_'].unique()

array(['No tiene', 'Sanidad ', 'Comercio y marketing', 'Imagen personal ',
       'Administración y gestión ',
       'Servicios socioculturales y a la comunidad ',
       'Estudios artísticos superiores (artes plásticas, diseño, música y danza, arte dramático o conservación y restauración de bienes culturales)',
       'Artes gráficas ', 'Edificación y obra civil ',
       'Hostelería y turismo ', 'Informática y comunicaciones',
       'Electricidad y electrónica',
       'Actividades físicas y deportivas (incluidas enseñanzas deportivas de grado superior)',
       'Química', 'Imagen y sonido', 'Artes y artesanías',
       'Textil, confección y piel', 'No conocido',
       'Seguridad y medio ambiente', 'Fabricación mecánica ',
       'Transporte y mantenimiento de vehículos ',
       'Instalación y mantenimiento', 'Industrias alimentarias ',
       'Marítimo-pesquera', 'Madera, mueble y corcho ', nan], dtype=object)

In [144]:
# Inspección:
df['fam_gsup3_'].unique()

array(['No tiene', 'Servicios socioculturales y a la comunidad ',
       'Imagen y sonido',
       'Estudios artísticos superiores (artes plásticas, diseño, música y danza, arte dramático o conservación y restauración de bienes culturales)',
       'Edificación y obra civil ', 'No conocido', 'Sanidad ',
       'Madera, mueble y corcho ', 'Electricidad y electrónica',
       'Informática y comunicaciones', 'Instalación y mantenimiento',
       'Química', 'Industrias alimentarias ',
       'Actividades físicas y deportivas (incluidas enseñanzas deportivas de grado superior)',
       'Comercio y marketing', 'Artes gráficas ', nan], dtype=object)

In [145]:
# Comprobación: todas las familias en la segunda y la tercera columnas originales están presentes también en ...
# ... la primera (permite tomar la primera como base para las agregaciones de familias)
df['fam_gsup2'].isin(df['fam_gsup1']).unique(), df['fam_gsup3'].isin(df['fam_gsup1']).unique()

(array([ True]), array([ True]))

In [146]:
# Variables binarias para cada familia de estudios de grado superior en cada posición (de 1 a 3):
df=pd.get_dummies(data=df,columns=['fam_gsup1_','fam_gsup2_','fam_gsup3_'],prefix=['fgs1','fgs2','fgs3'])

In [147]:
# Renombra las columnas resultantes de get_dummies prescindiendo de los textos entre paréntesis: 
D_fix=dict()
for col in list(filter(lambda col: col.startswith('fgs'),df.columns)): 
    D_fix[col]=col.split('(')[0].strip()
df.rename(columns=D_fix,inplace=True)

In [148]:
# Se prescinde de las columnas asociadas a valores 0 y 1 ('No tiene' o 'No conocido')
del df['fgs1_No tiene'], df['fgs2_No tiene'], df['fgs3_No tiene'], df['fgs1_No conocido']
try: df['fgs1_No conocido']
except: pass
try: df['fgs1_No conocido']
except: pass

In [149]:
# Lista de columnas binarias asociadas a familias de estudios de grado superior en la columna 'fam_gsup1'...
# ... (contiene todas las familias registradas):
lista_fam_gsup_1=list(filter(lambda col: col.startswith('fgs1'),df.columns))   
lista_fam_gsup_1[:5]

['fgs1_Actividades físicas y deportivas',
 'fgs1_Administración y gestión',
 'fgs1_Artes gráficas',
 'fgs1_Artes y artesanías',
 'fgs1_Comercio y marketing']

In [150]:
# Lista de familias de estudios de grado superior: 
lista_familias=list(map(lambda familia: familia.split('_')[1],lista_fam_gsup_1))
lista_familias[:5]

['Actividades físicas y deportivas',
 'Administración y gestión',
 'Artes gráficas',
 'Artes y artesanías',
 'Comercio y marketing']

In [151]:
# Se recogen los valores asociados a la misma familia en todas las columnas (hasta 3) asociadas a la misma: 
for familia in lista_familias:
    df['g_superior: '+str(familia)]=df['fgs1_'+str(familia)]
    if 'fgs2_'+str(familia) in df.columns: 
        df['g_superior: '+str(familia)]=df['g_superior: '+str(familia)]+df['fgs2_'+str(familia)]
    if 'fgs3_'+str(familia) in df.columns: 
        df['g_superior: '+str(familia)]=df['g_superior: '+str(familia)]+df['fgs3_'+str(familia)]

In [152]:
col_familias_gsup=list(filter(lambda col: re.findall('g_superior: ',col),df.columns))
col_familias_gsup[:5]

['g_superior: Actividades físicas y deportivas',
 'g_superior: Administración y gestión',
 'g_superior: Artes gráficas',
 'g_superior: Artes y artesanías',
 'g_superior: Comercio y marketing']

In [153]:
# Inspección: 
df[col_familias_gsup].agg(['value_counts'])

,g_superior: Actividades físicas y deportivas,g_superior: Administración y gestión,g_superior: Artes gráficas,g_superior: Artes y artesanías,g_superior: Comercio y marketing,g_superior: Edificación y obra civil,g_superior: Electricidad y electrónica,g_superior: Energía y agua,g_superior: Estudios artísticos superiores,g_superior: Fabricación mecánica,g_superior: Hostelería y turismo,g_superior: Imagen personal,g_superior: Imagen y sonido,g_superior: Industrias alimentarias,g_superior: Informática y comunicaciones,g_superior: Instalación y mantenimiento,"g_superior: Madera, mueble y corcho",g_superior: Marítimo-pesquera,g_superior: Química,g_superior: Sanidad,g_superior: Seguridad y medio ambiente,g_superior: Servicios socioculturales y a la comunidad,"g_superior: Textil, confección y piel",g_superior: Transporte y mantenimiento de vehículos,g_superior: Vidrio y cerámica
,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts
0,73160,72935.0,73417,73493.0,73304.0,73322,73211,73508.0,73078,73427.0,73309.0,73448.0,73262.0,73474.0,73115,73466.0,73508.0,73484.0,73380.0,72695,73443.0,72649.0,73500.0,73459.0,73509.0
1,342,572.0,87,20.0,207.0,181,291,5.0,401,82.0,201.0,64.0,247.0,39.0,381,47.0,5.0,29.0,131.0,780,70.0,846.0,13.0,54.0,4.0
2,10,6.0,8,NaN,2.0,8,9,NaN,27,4.0,3.0,1.0,4.0,NaN,16,NaN,NaN,NaN,2.0,37,NaN,18.0,NaN,NaN,NaN
3,1,NaN,1,NaN,NaN,2,2,NaN,7,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN


In [154]:
# Comprobación: 
check=cuenta_repeticiones(df,['fam_gsup1','fam_gsup2','fam_gsup3'])
check.rename(columns=D_dict['fam_gsup1'])

,No tiene,Agraria,Marítimo-pesquera,Industrias alimentarias,Química,Imagen personal,Sanidad,Seguridad y medio ambiente,Fabricación mecánica,Instalación y mantenimiento,Electricidad y electrónica,Energía y agua,Transporte y mantenimiento de vehículos,Edificación y obra civil,Vidrio y cerámica,"Madera, mueble y corcho","Textil, confección y piel",Artes gráficas,Imagen y sonido,Informática y comunicaciones,Administración y gestión,Comercio y marketing,Servicios socioculturales y a la comunidad,Hostelería y turismo,Actividades físicas y deportivas (incluidas enseñanzas deportivas de grado superior),Artes y artesanías,"Estudios artísticos superiores (artes plásticas, diseño, música y danza, arte dramático o conservación y restauración de bienes culturales)",nan
Repetitions,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,30416,73314,73484,73474,73380,73448,72695,73443,73427,73466,73211,73508,73459,73322,73509,73508,73500,73417,73262,73115,72935,73304,72649,73309,73160,73493,73078,43134
1,330,192,29,39,131,64,780,70,82,47,291,5,54,181,4,5,13,87,247,381,572,207,846,201,342,20,401,0
2,4890,6,0,0,2,1,37,0,4,0,9,0,0,8,0,0,0,8,4,16,6,2,18,3,10,0,27,0
3,37877,1,0,0,0,0,1,0,0,0,2,0,0,2,0,0,0,1,0,1,0,0,0,0,1,0,7,30379


In [155]:
# Actualiza listas de columnas auxiliares para eliminarlas: 
lista_fam_gsup_1=list(filter(lambda col: col.startswith('fgs1'),df.columns))  
lista_fam_gsup_2=list(filter(lambda col: col.startswith('fgs2'),df.columns)) 
lista_fam_gsup_3=list(filter(lambda col: col.startswith('fgs3'),df.columns)) 

In [156]:
# Se prescinde de columnas auxiliares: 
for col in lista_fam_gsup_1+lista_fam_gsup_2+lista_fam_gsup_3:
    del df[col]

In [157]:
# Se prescinde de columnas originales referidas a grados medios y superiores: 
obsoletas=list(filter(lambda col: re.findall('fam',col),df.columns))
obsoletas=[col for col in obsoletas if col not in filter(lambda col: re.findall('familia',col),df.columns)]
for col in obsoletas:
    del df[col]

In [158]:
# Actualización de diccionarios: 
for familia in lista_familias: 
    D_describe['g_superior: '+str(familia)]='Núm. de títulos de la familia de estudios de grado superior '\
    +str(familia)
    #print (D_describe['g_superior: '+str(familia)])

## Ajuste variables asociadas a ámbitos de estudio de grado universitario

Se agrupan todos los ámbitos de estudio de los títulos universitarios de grado en **variables ordinales**, de modo que las columnas resultantes recogen el número de títulos en cada ámbito en los que está titulado el encuestado. 

Para los encuestados de la encuesta de grado de 2014 solo se dispone de información sobre el ámbito de estudio del título principal (el título de grado que los hace elegibles para la encuesta)

In [159]:
# Inspección: 
df[['ambito_g1','ambito_g1_']].groupby('ambito_g1').agg(['value_counts'])

,,ambito_g1_
,,value_counts
ambito_g1,ambito_g1_,
0.0,No tiene,81
1.0,No conocido,331
11.0,Educación (Otros estudios),3913
21.0,Artes (Otros estudios),1976
22.0,Humanidades,2337
23.0,Lenguas,3053
31.0,Ciencias sociales y del comportamiento (Otros estudios),1691
32.0,Periodismo y documentación,1677


In [160]:
# Inspección:
df['ambito_g4_'].unique()

array(['No tiene', 'Formación de docentes de enseñanza primaria ',
       'Educación (Otros estudios) ',
       'Formación de docentes de enseñanza infantil ', 'No conocido',
       'Artes (Otros estudios) ', 'Actividades físicas y deportivas ',
       'Lenguas ', 'Humanidades ', 'Periodismo y documentación ',
       'Ciencias sociales y del comportamiento (Otros estudios) ',
       'Dirección y administración ', 'Enfermería ', 'Derecho ',
       'Industria manufacturera y producción ',
       'Negocios y administración (Otros estudios) ', 'Economía ',
       'Ingeniería y profesiones afines ', 'Informática',
       'Arquitectura y construcción ', 'Silvicultura ',
       'Matemáticas y estadística ', 'Servicios (Otros estudios) ',
       'Salud (Otros estudios) ', nan], dtype=object)

In [161]:
# Comprobación: todos los ámbitos de estudio relevantes de las titulaciones de grado están presentes en la ...
# ... columna referida al primer grado universitario (permite tomarla como base para construir el algoritmo de ...
# ... agregación de ámbitos de grado).
df['ambito_g2'].isin(df['ambito_g1']).unique(), df['ambito_g3'].isin(df['ambito_g1']).unique(),\
 df['ambito_g4'].isin(df['ambito_g1']).unique()

(array([ True, False]), array([ True, False]), array([ True, False]))

In [162]:
# La categoría presente en 'ambito_g4' que no está recogida en 'ambito_g1' son no informados. 
df['ambito_g4'][~df['ambito_g4'].isin(df['ambito_g1'])].unique()

array([nan])

In [163]:
# Inspección: El criterio para agrupar las titulaciones de máster en ámbitos de estudio es distinto del ...
# ... aplicado a los títulos de grado. 
df['ambito_m1_'].unique()

array([nan, 'SALUD Y SERVICIOS SOCIALES', 'EDUCACIÓN', 'INFORMÁTICA',
       'CIENCIAS', 'INGENIERÍA, INDUSTRIA Y CONSTRUCCIÓN',
       'NEGOCIOS, ADMINISTRACIÓN Y DERECHO', 'SERVICIOS',
       'CIENCIAS SOCIALES, PERIODISMO Y DOCUMENTACIÓN',
       'ARTES Y HUMANIDADES',
       'AGRICULTURA, GANADERÍA, SILVICULTURA, PESCA Y VETERINARIA'],
      dtype=object)

In [164]:
# Variables binarias para cada ámbito de estudios de grado en cada posición (de 1 a 4):

# Se renombran las variables literales para conservarlas en el proceso de codificación binaria: 
ambitos_g_u=list(filter(lambda col: col.startswith('ambito_g'),df.columns))
ambitos_g_u=list(filter(lambda col: re.findall('g[0-9]_',col),ambitos_g_u))
print ('ambitos_g_u',ambitos_g_u)
ambitos_temp=list(map(lambda col:'temp_'+col,ambitos_g_u))
print ('ambitos_temp',ambitos_temp)
for tupla in zip(ambitos_g_u,ambitos_temp): 
    df[tupla[1]]=df[tupla[0]]

df=pd.get_dummies(data=df,columns=ambitos_temp,prefix=['ag1','ag2','ag3','ag4'])

ambitos_g_u ['ambito_g1_', 'ambito_g2_', 'ambito_g3_', 'ambito_g4_']
ambitos_temp ['temp_ambito_g1_', 'temp_ambito_g2_', 'temp_ambito_g3_', 'temp_ambito_g4_']


In [165]:
# Renombra las columnas resultantes de get_dummies prescindiendo de los textos entre paréntesis: 
D_fix=dict()
for col in list(filter(lambda col: col.startswith('ag'),df.columns)): 
    D_fix[col]=col.split('(')[0].strip()
df.rename(columns=D_fix,inplace=True)

In [166]:
# Se prescinde de las columnas asociadas a valores 0 y 1 ('No tiene' o 'No conocido')
del df['ag1_No tiene'], df['ag2_No tiene'], df['ag3_No tiene'], df['ag4_No tiene'], df['ag1_No conocido']
try: df['ag2_No conocido']
except: pass
try: df['ag3_No conocido']
except: pass
try: df['ag4_No conocido']
except: pass

In [167]:
# Lista de columnas binarias asociadas a ámbitos de conocimiento de los títulos de grado en la columna ...
# ... 'ambito_g1' (contiene todos los ámbitos observados):
col_ambitos_g=list(filter(lambda col: col.startswith('ag1'),df.columns))   
col_ambitos_g[:5]

['ag1_Actividades físicas y deportivas',
 'ag1_Agricultura y ganadería',
 'ag1_Arquitectura y construcción',
 'ag1_Artes',
 'ag1_Ciencias de la vida']

In [168]:
# Lista de ámbitos de conocimiento de grado: 
lista_ambitos_g=list(map(lambda familia: familia.split('_')[1],col_ambitos_g))
lista_ambitos_g[:5]

['Actividades físicas y deportivas',
 'Agricultura y ganadería',
 'Arquitectura y construcción',
 'Artes',
 'Ciencias de la vida']

In [169]:
# Se recogen los valores asociados al mismo ámbito de estudios de grado en todas las columnas (hasta 4)
for ambito in lista_ambitos_g:
    df['n_g_ambito '+str(ambito)]=df['ag1_'+str(ambito)]
    if 'ag2_'+str(ambito) in df.columns: 
        df['n_g_ambito '+str(ambito)]=df['n_g_ambito '+str(ambito)]+df['ag2_'+str(ambito)]
    if 'ag3_'+str(ambito) in df.columns: 
        df['n_g_ambito '+str(ambito)]=df['n_g_ambito '+str(ambito)]+df['ag3_'+str(ambito)]
    if 'ag4_'+str(ambito) in df.columns: 
        df['n_g_ambito '+str(ambito)]=df['n_g_ambito '+str(ambito)]+df['ag4_'+str(ambito)]

In [170]:
col_n_g_ambito=list(filter(lambda col: re.findall('n_g_ambito ',col),df.columns))
col_n_g_ambito[:5]

['n_g_ambito Actividades físicas y deportivas',
 'n_g_ambito Agricultura y ganadería',
 'n_g_ambito Arquitectura y construcción',
 'n_g_ambito Artes',
 'n_g_ambito Ciencias de la vida']

In [171]:
# Inspección: 
df[col_n_g_ambito].agg(['value_counts'])

,n_g_ambito Actividades físicas y deportivas,n_g_ambito Agricultura y ganadería,n_g_ambito Arquitectura y construcción,n_g_ambito Artes,n_g_ambito Ciencias de la vida,"n_g_ambito Ciencias químicas, físicas y geológicas",n_g_ambito Ciencias sociales y del comportamiento,n_g_ambito Derecho,n_g_ambito Dirección y administración,n_g_ambito Economía,n_g_ambito Educación,n_g_ambito Enfermería,n_g_ambito Formación de docentes de enseñanza infantil,n_g_ambito Formación de docentes de enseñanza primaria,n_g_ambito Humanidades,n_g_ambito Industria manufacturera y producción,n_g_ambito Informática,n_g_ambito Ingeniería y profesiones afines,n_g_ambito Lenguas,n_g_ambito Matemáticas y estadística,n_g_ambito Medicina,n_g_ambito Medio ambiente,n_g_ambito Negocios y administración,n_g_ambito Periodismo y documentación,n_g_ambito Psicología,n_g_ambito Salud,n_g_ambito Servicios,n_g_ambito Servicios sociales,n_g_ambito Silvicultura,n_g_ambito Técnicas audiovisuales y medios de comunicación,n_g_ambito Veterinaria,"n_g_ambito Viajes, turismo y ocio"
,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts
0,71963.0,72034.0,69546,71448.0,70960.0,71082.0,71576.0,69588.0,67802.0,71729.0,68699.0,70167.0,70353.0,70014,70973.0,72244.0,70538.0,67214,70315,72446.0,71839.0,72350.0,70891.0,71696.0,71306.0,69945,72780.0,71664.0,72878.0,72326.0,72585.0,71873.0
1,1544.0,1307.0,3394,2022.0,2468.0,2415.0,1880.0,3911.0,5210.0,1781.0,4347.0,3229.0,3077.0,3310,2495.0,1180.0,2667.0,5492,3054,988.0,1672.0,1163.0,2599.0,1752.0,2201.0,3404,709.0,1738.0,559.0,1185.0,922.0,1588.0
2,6.0,166.0,548,41.0,85.0,15.0,56.0,14.0,486.0,3.0,443.0,117.0,82.0,178,43.0,84.0,300.0,769,138,78.0,2.0,NaN,23.0,64.0,6.0,152,24.0,110.0,76.0,2.0,6.0,50.0
3,NaN,6.0,24,2.0,NaN,1.0,1.0,NaN,15.0,NaN,24.0,NaN,1.0,9,2.0,5.0,8.0,35,5,1.0,NaN,NaN,NaN,1.0,NaN,10,NaN,1.0,NaN,NaN,NaN,2.0
4,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,3,1,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN


In [172]:
# Comprobación: 
check=cuenta_repeticiones(df,['ambito_g1','ambito_g2','ambito_g3','ambito_g4'])
check.rename(columns=D_dict['ambito_g1'])

,No tiene,No conocido,Educación (Otros estudios),Medicina,Enfermería,Artes (Otros estudios),Humanidades,Lenguas,Dirección y administración,Ciencias sociales y del comportamiento (Otros estudios),Periodismo y documentación,Negocios y administración (Otros estudios),Derecho,Ciencias de la vida,Medio ambiente,"Ciencias químicas, físicas y geológicas",Matemáticas y estadística,Economía,Psicología,Informática,Ingeniería y profesiones afines,Industria manufacturera y producción,Arquitectura y construcción,Agricultura y ganadería,Silvicultura,Técnicas audiovisuales y medios de comunicación,Veterinaria,Salud (Otros estudios),Servicios sociales,Servicios (Otros estudios),Formación de docentes de enseñanza infantil,Formación de docentes de enseñanza primaria,Actividades físicas y deportivas,"Viajes, turismo y ocio",nan
Repetitions,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,30756,73083,68699,71839,70167,71448,70973,70315,67802,71576,71696,70891,69588,70960,72350,71082,72446,71729,71306,70538,67214,72244,69546,72034,72878,72326,72585,69945,71664,72780,70353,70014,71963,71873,31651
1,2949,399,4347,1672,3229,2022,2495,3054,5210,1880,1752,2599,3911,2468,1163,2415,988,1781,2201,2667,5492,1180,3394,1307,559,1185,922,3404,1738,709,3077,3310,1544,1588,11483
2,15498,27,443,2,117,41,43,138,486,56,64,23,14,85,0,15,78,3,6,300,769,84,548,166,76,2,6,152,110,24,82,178,6,50,0
3,24310,4,24,0,0,2,2,5,15,1,1,0,0,0,0,1,1,0,0,8,35,5,24,6,0,0,0,10,1,0,1,9,0,2,30379
4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,3,0,1,0,0,0,0,2,0,0,0,2,0,0,0


In [173]:
# Actualiza listas de columnas auxiliares para eliminarlas: 
lista_ambito_1=list(filter(lambda col: col.startswith('ag1_'),df.columns))  
lista_ambito_2=list(filter(lambda col: col.startswith('ag2_'),df.columns)) 
lista_ambito_3=list(filter(lambda col: col.startswith('ag3_'),df.columns)) 
lista_ambito_4=list(filter(lambda col: col.startswith('ag4_'),df.columns)) 

In [174]:
# Se prescinde de columnas auxiliares: 
for col in lista_ambito_1+lista_ambito_2+lista_ambito_3+lista_ambito_4:
    del df[col]

In [175]:
# Actualización de diccionarios: 
for ambito in lista_ambitos_g: 
    D_describe['n_g_ambito '+str(ambito)]='Núm. de títulos de grado del ámbito de estudios '\
    +str(ambito)
    #print (D_describe['n_g_ambito '+str(ambito)])

## Simplificación de sectores CNAE

In [176]:
# Diccionario de sectores cnae para interpretación de resultados de regresión logística:
df_oc=df[df['ocupacion_'].notna()]
D_cnae=dict(zip(df_oc['cnae'].unique(),df_oc['cnae_'].unique()))
D_cnae

{23: 'Educación',
 26: 'Actividades asociativas  y sindicales',
 24: 'Actividades sanitarias y de servicios sociales',
 18: 'Actividades de agencias de viajes, operadores turísticos, servicios de reservas y actividades relacionadas con los mismos',
 21: 'Actividades administrativas de oficina y otras actividades auxiliares a las empresas',
 13: 'Actividades profesionales, científicas y técnicas',
 1: 'Agricultura, ganadería, silvicultura y pesca',
 9: 'Hostelería',
 22: 'Administración Pública y defensa; Seguridad Social obligatoria',
 14: 'Actividades de fotografía',
 31: 'Actividades de organizaciones y organismos extraterritoriales',
 7: 'Comercio al por mayor y al por menor; reparación de vehículos de motor y motocicletas',
 20: 'Servicios a edificios y actividades de jardinería',
 25: 'Actividades artísticas, culturales, recreativas, deportivas y de entrenimiento',
 17: 'Actividades relacionadas con el empleo',
 3: 'Industria manufacturera',
 8: 'Transporte, almacenamiento, activi

In [181]:
D_cnae[18]='Agencias de viajes, operadores turísticos, etc.'
D_cnae[21]='Actividades administrativas de oficina y otras auxiliares'
D_cnae[22]='Administración Pública y defensa, Seguridad Social obligatoria'
D_cnae[7]='Comercio y reparación de vehículos'
D_cnae[25]='Actividades artísticas, culturales, deportivas, etc.'
D_cnae[10]='Información y comunicación (TIC, comunicación audiovisual, etc.)'
D_cnae[28]='Servicios personales y de conservación de artículos de uso doméstico'
D_cnae[30]='Particulares como empleadores de personal doméstico o productores para autoconsumo'
D_cnae[5]='Suministro de agua, saneamiento, gestión de residuos y descontaminación'
D_cnae[0]='No conocido'

In [182]:
df['cnae_']=df['cnae'].replace(D_cnae)
df['cnae_'].unique() # Comprobación

array(['Educación', 'No conocido',
       'Actividades asociativas  y sindicales',
       'Actividades sanitarias y de servicios sociales',
       'Agencias de viajes, operadores turísticos, etc.',
       'Actividades administrativas de oficina y otras auxiliares',
       'Actividades profesionales, científicas y técnicas',
       'Agricultura, ganadería, silvicultura y pesca', 'Hostelería',
       'Administración Pública y defensa, Seguridad Social obligatoria',
       'Actividades de fotografía',
       'Actividades de organizaciones y organismos extraterritoriales',
       'Comercio y reparación de vehículos',
       'Servicios a edificios y actividades de jardinería',
       'Actividades artísticas, culturales, deportivas, etc.',
       'Actividades relacionadas con el empleo',
       'Industria manufacturera',
       'Transporte, almacenamiento, actividades postales y de correos',
       'Actividades inmobiliarias', 'Actividades veterinarias',
       'Suministro de energía eléctri

## Archivo csv

In [183]:
df.to_csv('data/formacion_procesado.csv')